In [239]:
%%time
dataset_name = "W19_comb"
df_list = [ "BES_Panel" ]

%matplotlib inline 
%run BES_header.py {dataset_name} {df_list}

if "id" in BES_Panel.columns:
    BES_Panel = BES_Panel.set_index("id").sort_index()

(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)    
    
# get full set of inferred "cross wave" auth-lib/left-right values and ages
pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')



Toggle code

var_type (7911, 14)
Wall time: 33.4 s


In [240]:
search(BES_Panel,"newspaper")

newspaperReadW15          30842
profile_newspaperW10      30237
profile_newspaperW11      30956
profile_newspaper2W12     34394
profile_newspaperW13      31136
profiles_newspaper2W16    13775
profiles_newspaper2W17    34366
profile_newspaper         56968
dtype: int64

In [247]:
BES_Panel = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES_Panelv02",compression='zip')

BES_Panel = BES_Panel.set_index("id")
BES_Panel = BES_Panel.sort_index()

Media variables
all these variables reflect the "profile_newspaper_readership_201" yougov variable

note that this is "Which daily newspaper do you read most often?"

you could legitimately read the Express and Mail about as often and oscillate between

also there's a huge editorial (and anecdotally I understand readership) difference between

Mail on Sunday (Remain! - contains weekly TV listings) and the Daily Mail (Leave!)

"profiles_newspaper2W16" is obviously just a "top up" sample

"profile_newspaper" is an amalgamation of all the waves

Full Waves 10, 11, 12, 13, 15

In [88]:
search(BES_Panel,"p_paper_read")
# beautiful!

p_paper_readW1     30383
p_paper_readW2     30239
p_paper_readW3     28122
p_paper_readW4     31442
p_paper_readW5     30829
p_paper_readW6     30149
p_paper_readW7     30841
p_paper_readW8     33500
p_paper_readW9     30034
p_paper_readW10    30236
p_paper_readW11    30955
p_paper_readW12    34393
p_paper_readW13    31135
p_paper_readW14    31061
p_paper_readW15    30840
p_paper_readW16    37959
p_paper_readW17    34365
p_paper_readW18    37824
p_paper_readW19    32176
dtype: int64

In [89]:
BES_Panel[search(BES_Panel,"p_paper_read").index[:-1]]

,p_paper_readW1,p_paper_readW2,p_paper_readW3,p_paper_readW4,p_paper_readW5,p_paper_readW6,p_paper_readW7,p_paper_readW8,p_paper_readW9,p_paper_readW10,p_paper_readW11,p_paper_readW12,p_paper_readW13,p_paper_readW14,p_paper_readW15,p_paper_readW16,p_paper_readW17,p_paper_readW18
id,,,,,,,,,,,,,,,,,,
1,The Mirror / Daily Record,The Mirror / Daily Record,NaN,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Daily Telegraph,NaN,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,NaN,NaN,None,None
3,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,NaN,NaN,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,None,None,NaN,NaN
4,The Sun,The Sun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The Daily Telegraph,The Daily Telegraph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
98448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
98449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [90]:
BES_Panel[search(BES_Panel,"p_paper_read").index].apply(lambda x: "|".join(x.cat.categories)).value_counts()
# consistent categories!

The Express|The Daily Mail / The Scottish Daily Mail|The Mirror / Daily Record|The Daily Star / The Daily Star of Scotland|The Sun|The Daily Telegraph|The Financial Times|The Guardian|The Independent|The Times|The Scotsman|The Herald (Glasgow)|The Western Mail|Other local daily morning newspaper|Other Newspaper|None    19
dtype: int64

In [91]:
BES_Panel["p_paper_readW18"].value_counts()

None                                           17321
The Daily Mail / The Scottish Daily Mail        5010
The Guardian                                    3645
The Sun                                         2156
The Times                                       1906
Other Newspaper                                 1858
The Daily Telegraph                             1728
The Mirror / Daily Record                       1091
Other local daily morning newspaper             1063
The Express                                      712
The Independent                                  685
The Financial Times                              220
The Daily Star / The Daily Star of Scotland      159
The Herald (Glasgow)                             127
The Scotsman                                      98
The Western Mail                                  45
Name: p_paper_readW18, dtype: int64

In [92]:
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=False,match=False,
                                   low_priority_values=["Other local daily morning newspaper","Other Newspaper","None"])

Amalgamating variables: 
{16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profiles_newspaper2W16', 'profiles_newspaper2W17']


In [93]:
newspaper_amalg.value_counts()

None                                           20626
The Daily Mail / The Scottish Daily Mail        4818
The Guardian                                    4163
The Sun                                         2233
The Times                                       1903
The Daily Telegraph                             1690
Other Newspaper                                 1584
The Mirror / Daily Record                       1136
Other local daily morning newspaper             1065
The Independent                                  931
The Express                                      671
The Financial Times                              285
The Daily Star / The Daily Star of Scotland      165
The Herald (Glasgow)                             129
The Scotsman                                      95
The Western Mail                                  45
Name: profiles_newspaper2W16&17, dtype: int64

In [94]:
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=False,match=False,
                                   low_priority_values=[])
newspaper_amalg.value_counts()

Amalgamating variables: 
{16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profiles_newspaper2W16', 'profiles_newspaper2W17']


None                                           20909
The Daily Mail / The Scottish Daily Mail        4726
The Guardian                                    4086
The Sun                                         2195
The Times                                       1877
The Daily Telegraph                             1664
Other Newspaper                                 1626
The Mirror / Daily Record                       1111
Other local daily morning newspaper             1096
The Independent                                  898
The Express                                      656
The Financial Times                              273
The Daily Star / The Daily Star of Scotland      162
The Herald (Glasgow)                             125
The Scotsman                                      91
The Western Mail                                  44
Name: profiles_newspaper2W16&17, dtype: int64

In [95]:
%%time
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=True,match=False,
                                   low_priority_values=[])
newspaper_amalg.value_counts()

Amalgamating variables: 
{16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profiles_newspaper2W16', 'profiles_newspaper2W17']
Wall time: 1min 34s


None                                           21051
The Daily Mail / The Scottish Daily Mail        4690
The Guardian                                    4068
The Sun                                         2172
The Times                                       1857
The Daily Telegraph                             1643
Other Newspaper                                 1614
Other local daily morning newspaper             1100
The Mirror / Daily Record                       1093
The Independent                                  898
The Express                                      650
The Financial Times                              294
The Daily Star / The Daily Star of Scotland      148
The Herald (Glasgow)                             123
The Scotsman                                      91
The Western Mail                                  47
Name: profiles_newspaper2W16&17, dtype: int64

In [96]:
# they're a bit None heavy (and region matters)

In [97]:
# BES_Panel["profile_newspaper"].value_counts()
### hmm - discrepancy alert

In [98]:
search(BES_Panel,"newspaper").index

Index(['newspaperReadW15', 'profiles_newspaper2W16', 'profiles_newspaper2W17'], dtype='object')

In [99]:
BES_Panel["newspaperReadW15"].value_counts()


None                                           15350
The Daily Mail / The Scottish Daily Mail        3715
The Guardian                                    2617
The Sun                                         1804
The Times                                       1415
Other Newspaper                                 1292
The Daily Telegraph                             1187
The Mirror / Daily Record                        956
Other local daily morning newspaper              843
The Independent                                  619
The Express                                      507
The Financial Times                              189
The Daily Star / The Daily Star of Scotland      140
The Herald (Glasgow)                              93
The Scotsman                                      72
The Western Mail                                  43
Name: newspaperReadW15, dtype: int64

In [100]:
# BES_Panel["profile_newspaperW10"].value_counts()

In [101]:
# BES_Panel["profile_newspaperW11"].value_counts()

In [102]:
# BES_Panel["profile_newspaper2W12"].value_counts()

In [103]:
# BES_Panel["profile_newspaperW13"].value_counts()

In [104]:
# BES_Panel["profiles_newspaper2W16"].value_counts()

In [105]:
# BES_Panel["profiles_newspaper2W17"].value_counts()

In [106]:
# (BES_Panel[search(BES_Panel,"newspaper2?W").index]=="The Sun").any(axis=1).sum()

## how come 2000 extra Sun readers just appear for profile_newspaper?

In [107]:

# BES_Panel[search(BES_Panel,"newspaper2?W").index].loc[BES_Panel["profile_newspaper"]=="The Sun"].dropna(axis=0,how='all')

In [108]:
# BES_Panel["profiles_newspaper2W17"][BES_Panel["profile_newspaper"]=="The Sun"].value_counts()

BES Media module variables
infoSource

During the last seven days, on average how much time (if any) have you spent per day following news about

politics or current affairs from each of these sources?

TV/Paper/Radio/Internet/People

Full waves 4, 5, 6, 7, 8, 12, 13, 15, 17, 18, 19

In [109]:
search(BES_Panel,"infoSource")

infoSourceTVW4           31552
infoSourcePaperW4        31552
infoSourceRadioW4        31552
infoSourceInternetW4     31552
infoSourcePeopleW4       31552
infoSourceTVW5           30969
infoSourcePaperW5        30969
infoSourceRadioW5        30969
infoSourceInternetW5     30969
infoSourcePeopleW5       30969
infoSourceTVW6           30257
infoSourcePaperW6        30257
infoSourceRadioW6        30257
infoSourceInternetW6     30257
infoSourcePeopleW6       30257
infoSourceTVW7           30895
infoSourcePaperW7        30895
infoSourceRadioW7        30895
infoSourceInternetW7     30895
infoSourcePeopleW7       30895
infoSourceTVW8           33502
infoSourcePaperW8        33502
infoSourceRadioW8        33502
infoSourceInternetW8     33502
infoSourcePeopleW8       33502
infoSourceTVW12          34394
infoSourcePaperW12       34394
infoSourceRadioW12       34394
infoSourceInternetW12    34394
infoSourceTVW13          31136
infoSourcePaperW13       31136
infoSourceRadioW13       31136
infoSour

In [110]:
BES_Panel["infoSourceTVW4"].value_counts()

Less than 1/2 hour      8861
1/2 hour to 1 hour      8585
1 to 2 hours            5334
None, no time at all    4199
More than 2 hours       3505
Don't know              1068
Name: infoSourceTVW4, dtype: int64

twitterUse: Full Waves 4, 5, 6, 12, 18

Do you use Twitter? Yes/No/DK (mostly No!, ~8000/30+k = Yes)

~8000 people that say Yes then asked

twitterInfo(_1/2/3) Partial Sample Waves 4, 5, 12, 18

In the last 4 weeks, have you read news or found information about the upcoming general election or politics

more generally that was tweeted by any of the following people or organisations? Tick all that apply

twitterInfo_1: Candidates or parties on Twitter

twitterInfo_2: Someone you know personally on Twitter

twitterInfo_3: Other people on Twitter e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on twitter (mostly Don't follow politics on twitter)

twitterInfof2f Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through twitter?
Yes/No/DK

In [111]:
search(BES_Panel,"twitter")

twitterUseW4        31551
twitterInfo_1W4      8799
twitterInfo_2W4      8799
twitterInfo_3W4      8799
twitterUseW5        30970
twitterInfo_1W5      8777
twitterInfo_2W5      8777
twitterInfo_3W5      8777
twitterUseW6        30257
twitterInfof2fW6     8440
twitterUseW12       34394
twitterInfo_1W12     9848
twitterInfo_2W12     9848
twitterInfo_3W12     9848
twitterUseW18       37825
twitterInfo_1W18    10807
twitterInfo_2W18    10807
twitterInfo_3W18    10807
dtype: int64

In [112]:
BES_Panel["twitterInfof2fW6"].value_counts()

Yes           4505
No            3821
Don't know     114
Name: twitterInfof2fW6, dtype: int64

fbUse: Full Waves 4, 5, 6, 12, 18

Do you use Facebook? Yes/No/DK (mostly No!, ~20k/30+k = Yes)

~20000 people that say Yes then asked

fbInfo(_1/2/3) Partial Sample Waves 4, 5, 12, 18

In the last 4 weeks, have you read news or found information about the upcoming general election or politics more

generally that was posted on Facebook by any of the following people or organisations? Tick all that apply

fbInfo_1 Candidates or parties on Facebook

fbInfo_2 Someone you know personally on Facebook

fbInfo_3 Other people on Facebook e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on facebook (mostly Don't follow politics on facebook - even moreso than twitter)

fbInfof2fW6 Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through Facebook?
Yes/No/DK

In [113]:
search(BES_Panel,"fb",case_sensitive=True)

fbUseW4        31551
fbInfo_1W4     21008
fbInfo_2W4     21008
fbInfo_3W4     21008
fbUseW5        30970
fbInfo_1W5     20632
fbInfo_2W5     20632
fbInfo_3W5     20632
fbUseW6        30258
fbInfof2fW6    20233
fbUseW12       34394
fbInfo_1W12    23538
fbInfo_2W12    23538
fbInfo_3W12    23538
fbUseW18       37825
fbInfo_1W18    25248
fbInfo_2W18    25248
fbInfo_3W18    25248
dtype: int64

In [114]:
BES_Panel["fbInfo_1W4"].value_counts()

Don't follow politics on facebook    14358
No                                    3038
Yes                                   2609
Don't know                            1003
Name: fbInfo_1W4, dtype: int64

euSources Full Waves 7, 8

Have you heard anything about the EU referendum from any of these during the last seven days?

Please tick all that apply.

euSources_1: Television

euSources_2: Newspaper (including online)

euSources_3: Radio

euSources_4: Internet (not including online newspapers)

euSources_5: Talking to other people

Yes/No/DK

In [115]:
search(BES_Panel,"euSources")

euSources_1W7      30895
euSources_2W7      30895
euSources_3W7      30895
euSources_4W7      30895
euSources_5W7      30895
euSources_111W7    30781
euSources_1W8      33502
euSources_2W8      33502
euSources_3W8      33502
euSources_4W8      33502
euSources_5W8      33502
euSources_111W8    33391
dtype: int64

In [116]:
BES_Panel["euSources_1W7"].value_counts()

Yes           19147
No             9568
Don't know     2180
Name: euSources_1W7, dtype: int64

In [522]:
# Which daily newspaper do you read most often?
df = BES_Panel[search(BES_Panel,"p_paper_read").index].copy()

# During the last seven days, on average how much time (if any) have you spent per day
# following news about politics or current affairs from each of these sources?
df[search(BES_Panel,"infoSource").index] = BES_Panel[search(BES_Panel,"infoSource").index]

# (fb|twitter)Use
# Do you use Facebook|Twitter?
# (fb|twitter)Info_1
# Candidates or parties on Twitter
# (fb|twitter)Info_2
# Someone you know personally on Twitter
# (fb|twitter)Info_3
# Other people on Twitter e.g. commentators, journalists, activists

df[match(BES_Panel,"fb|twitter",case_sensitive=True).index] = BES_Panel[match(BES_Panel,"fb|twitter",case_sensitive=True).index]

# euSources
# Have you heard anything about the EU referendum from any of these during the last seven days?
# euSources_1
# Television
# euSources_2
# Newspaper (including online)
# euSources_3
# Radio
# euSources_4
# Internet (not including online newspapers)
# euSources_5
# Talking to other people

df[match(BES_Panel,"euSources",case_sensitive=True).index] = BES_Panel[match(BES_Panel,"euSources",case_sensitive=True).index]

In [258]:
df.columns

Index(['p_paper_readW1', 'p_paper_readW2', 'p_paper_readW3', 'p_paper_readW4',
       'p_paper_readW5', 'p_paper_readW6', 'p_paper_readW7', 'p_paper_readW8',
       'p_paper_readW9', 'p_paper_readW10',
       ...
       'euSources_3W7', 'euSources_4W7', 'euSources_5W7', 'euSources_111W7',
       'euSources_1W8', 'euSources_2W8', 'euSources_3W8', 'euSources_4W8',
       'euSources_5W8', 'euSources_111W8'],
      dtype='object', length=118)

In [260]:
df.to_stata('../BES_analysis_data/W19_comb_strings\\BES_media_vars.dta')
df.to_csv('../BES_analysis_data/W19_comb_strings\\BES_media_vars.csv')

In [529]:
df_temp = pd.get_dummies( df.drop(['euSources_111W7', 'euSources_111W8'],axis=1) )

df_temp = df_temp.drop([x for x in df_temp.columns if "Don't know" in x],axis=1)
df_temp

,p_paper_readW1_The Express,p_paper_readW1_The Daily Mail / The Scottish Daily Mail,p_paper_readW1_The Mirror / Daily Record,p_paper_readW1_The Daily Star / The Daily Star of Scotland,p_paper_readW1_The Sun,p_paper_readW1_The Daily Telegraph,p_paper_readW1_The Financial Times,p_paper_readW1_The Guardian,p_paper_readW1_The Independent,p_paper_readW1_The Times,...,euSources_1W8_No,euSources_1W8_Yes,euSources_2W8_No,euSources_2W8_Yes,euSources_3W8_No,euSources_3W8_Yes,euSources_4W8_No,euSources_4W8_Yes,euSources_5W8_No,euSources_5W8_Yes
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,1,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [530]:
df_temp.to_stata('../BES_analysis_data/W19_comb_strings\\BES_media_vars_dummied.dta')
df_temp.to_csv(  '../BES_analysis_data/W19_comb_strings\\BES_media_vars_dummied.csv')

need to download/open up the open-ended question dataset for
tvWatch/radioList/paperRead/internetRead

urk - ugly human data

What are the most important Internet sources (including online newspapers) you read

for information about politics and the election?

List up to three websites or click next if there are no websites you can think of.

TV/radio/newspaper/internet

whole bunch of variables about visiting a party site/sharing party website/fb/twitter stuff -> ignoring

In [355]:
df=pd.read_pickle('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.3.zip', compression='zip')

In [470]:
crap_list = ["__na__","none","dk","__dk__","na","0","?",".","don't know","dont know","-","d/k","n/a","o","n",
             "x","dn","no","n.a","a","not sure","1","2","3","4","5","6","7","8","9","non","nil","no tv","no idea",
             "unsure","s","/",","," none","n/c","don't read newspapers","n a","nothing","h","g","any",
             "i don't read newspapers","f","e","all","c","??","z","l","v","???","q","b","i don't know","idk","do not read",
             "pass","various","m","do not know","an","as above","..","not applicable","d",",",
             'k', "'", '\\', 'y', 'w', '"', '=', 'r', 'j', 'u', 't', 'p', '`', ';', '>', '+', '!',
             '--','00','.?','//',',,',"''",'>?','?/','@?',"'i",'p<','/.',".'",'>/',',l','n/',':?',' 1',
             '...','///','---','...','n\\a','n?a','n,a','000',
             'n/a.','....','////','????','i dk','any.','n./a','n.,a',' n/a','`n.a','..?.','`n/a','in/a','-///',
             'n.aq','n/a/','----','n /a','nm/a',
             '?????','no ne','n.?/a','-----','.....','/////','none-','nione','/////','n.a..','.none','noene', ',,,,,',
             "can't",'non e', '>>>>>', "do 't",'"non"','nmone',
             "don't",'?????', 'nnone', 'none3', 'onone', 'nonet','n.?/a','none`','arsed','noned','-----','.....',
             'nnoen','none-','nione','/////','n.a..','noner',',,,,,',"can't",'non e','>>>>>',"do 't",'"non"','nmone',
             'xxxxxxx','xxxxxx','xxxxx','xxxx','xxx','xx','x',
             "i don't",'notsure','any/all','did not','noneone','i avoid',
             "i didn't",'whatever', 'no telly', 'i do not', 'your mum','dontread','read the', 'dont use',
             'fkuyhhjh','xcvxcvxc','kjgmfdgf','dddssdsd','jgcvhbmn','xcvcxvxc','tyyuhjkl','wsrfgwrg','ghhhgfgj',
             'hjfcgchg', 'ahsjsjsj','dontwatch','nonr','dont','d k','none','i dont','no e','noine','noe',"i haven't", '"',
             'ditto','dont read','jjj', 'iii', 'jj','djajsjajs','jyfjhjm','don tknow',
             'not sure', 'vznfndndk', 'no paper','sss',"don't kbo",'sdvsdf','in/a','ghgfgg',';iluluyfg','gcfhd',
             'iiii',"don't kno",'not known',"don't kno", 'na',
             'scvxdv','czdvfb', 'rghwrghwr','shsjjs','chifyj', 'xfckkj','fhhjkkkkk','ghsgfshsf','wrgwrhg','i didnt','werhwrhwr',
             'i have no tv','"', 'u', 'n', 'd',
             'do not watch tv',"don't watch","no radio","don't listen to radio","none","nobe","same",
             "radio","can't remember","next","nk","dont listen to the radio","good","npne",
             "not one","know","listen","noen","do not listen to radio","no other","do not read newspapers","nine",
             "dont read newspapers","varies","nill","no newspaper","there is no trustworthy newspaper","i do not read newspapers",
             "no scooby doo","not","all are biased","nothing specific","hey","not read","stop","lies lies lies on waste paper","out of uk at present",
             "never","away on hol","did not read","i don not read politics in newspapers","havent read a newspaper","don't read","n/a don't buy papers",
             "i don't pay for second hand views of the illuminati","read","unknown","there are none","above",
             "not important","on holiday","none in print","d/n","its ok","none read","not read any","dont no","above",
             '"','11','12','42','n0','u','n/o','yes','n.a','n/k', 'm/a','na2', 'nk;', 'd.k',"n'a",
             "any news","none2","other","dont listen to radio","general radio","do not use","don't know any",
             "no websites","non biased","been on a cruise","don't mnkw","cant remember","did not watch",
             "don't watch tv","don't watch tv","i don't have a tv","i don't watch tv","tv","watch","can't recall",
             "do not watch any tv",
            "i don't own a television","errtff","chdn","i don't accept this matter arises",
            "not sute","don;t know","all biased",
            "iuiuiokolp","cant","jfdnf","xccxcxcxcx","dont know 2", "98","forget",
            "advertising","don't have tv in the house","dont watch tv programmes about news or politics","didn't",
            "not iimportsnt","don't mr know","assorted","as i'm on holiday",
            "typo","our tv has got an off button","01383Â 730050",
            "i never said i watched live tv", 
            "didnt watch","don't jniw","okkk","yuuijjkkkkkm","dontknow","vccvvc","hsbababdg","oooo",
            "dont known","don t watch","not interested","as before","sure"
            "i dont watch tv","dont knowd","not applicable on holiday","dont watch any",
            "first tv programme","dkn","i don't watch tv politics","not watching","n/a no tv",
            "programmes i can't remember","too biased","don t know","nont",
            "i don't watch tv at home","none i can think of don't remember","i do not watch tv",
            "i can","none live  no tv","none. on holiday","dont watch t v","dont watch  tv","10",
            "couldn't be bothered","not watch","why","dont have a tv","dont remember",
            "don't follow politics at all","didn't watch any","did not watch tv","no others",
            "did not watch any tv last week.  did not have time","n/a - out of uk at present",
            "havent watched television","tv cannot be trusted","i do not watch political programmes",
            "can think of","i dont own a tv","none","they're all biased","not watched",
            "none i have been abroad","dont know 3","dint know",
            "i already told you i don't watch tv","none they r biased",
            "i have been away on holiday","so i have not seen any tv",
            "none, this has been going on since bloody xmas, i am losing the will to live",
            "i try to avoid all politics progs including the news",
            "i don't watch tv anymore it's to biased",
            "ticked this in error, sorry","f knows",
            "do not watch tv as biased","not sure of names",
            "whoever stole the off switch for my mind please return it, no questions asked",
            "none i would rather watch paint dry",
            "tv programmes are bought and paid for by elites",
            "i don't watch television","none, all have proven to be biased",
            "none because i have made my mind up and i am fed up with some of the antics of the polititians",
            "none - was on holiday","i've been on holiday so not watched uk tv",
            "i don't watch plitical programmes","no interest in politics","80",
            "dont watch politics programmes as they are all just mouth pices for the labour party",
            "voted so not watching","i don't watch much tv","n/a rarely watch tv","do not watch as biased",
            "no interest in party politics","don't watch live tv","n.a. i do not have a television",
            "didn't watch any tv","dont watch tv","did not watch any","don't have a tv","cannot remember",
            "bbc are too biases","cant mind","n/a rarely watchtv","none, fed up with the lot of them",
            "one of our local news programmes with the conservative candidate and the labour candidate. labour was awful",
            "none - was on holiday in italy","what","dont follow politics too closely","seriously, tv is obnoxious",
            "tv is so biased it is not worth watching",
            "none on holiday",
            "been on holiday",
            "none, been on holiday",
            "none - on holiday",
            "on holiday abroard",
            "i have been away on holiday, so i have not seen any tv",
            "none been on holiday",
            "none, they're all biased",
            "don't have tv",
            "don't have access to a tv",
            "don't knoe",
            "don't kniw",
            "don't habe tv",
            "don't  know",
            "don't remember",
            "i don't watch any",
            "i have given up on the bbc",
            "dont know what called",
            "dn't know",
            "don't  know",
            "don.t know",
            "cant recall",
            "cant rememeber",
            "cant remeber",
            "can't remember  name of show",
            "can't think",
            "can't remember the name",
            "itv @ 12:30 can't rem name",
            "whatevers on",
            "see above",
            "not got radio",
            "no particular prgrams",
            "none am deaf",
            "vyggh",
            "201",
            "i dont know",
            "what the",
            "not listened to radio in 7 days",
            "i made a mistake",
            "dont usually listen to radio",
            "not sure 2",
            "nonw",
            "none n",
            "no ideano idea",
            "oioojjjkkkll",
            "wgngmg",
            "nonne",
            "didn't listen to radio at all",
            "else",
            "i still don't",
            "can't remember name",
            "n/a -out of uk at present",
            "i ddon;t listen to political radio",
            "haven't listened to another programme",
            "kllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll",
            "don't listen to the radio either",
            "cant rememebr",
            "yuuiioijokkl",
            "none spring to mind",
            "didnt listen",
            "yygggh",
            "no other station",
            "nothing here unbiased and worth listening to",
            "mone",
            "do not listen to radio so i don't know",
            "fo not know",
            "dont know of any",
            "been away",
            "nil - i do not listen to the radio",
            "i don't listen to radio",
            "do t listen",
            "gjhgg",
            "donr k now",
            "not surenot sure",
            "none didnt have radio on",
            "dont listen to radio at all",
            "above are the only two stations i listen to",
            "nons",
            "xxxxxxxxxxxxx",
            "don't listen to any",
            "don't listen to the radio",
            "font listed to rafio",
            "to any",
            "fbnngb",
            "no radio in house",
            "no programs",
            "don';t know",
            "i have been away",
            "xxxxxxxxxxxxxxx","n0n4",
            "nznxncmcm","ghjjj","cxvxcxc","jydb","wrgwr","ffvvfvfvvfvfv","jjaeÂ·kkjjjjkjkj","xkajxjh",
            "i do not listen on the radio to political programmes",
            "don't know title",
            "too",
            "it varies depends when i am driving",
            "n/a-partially deaf",
            "didnt listen to radio",
            "don't remember",
            "if on i listen other wise no special programme",
            "n/a - sometimes hear it by chance if i am out and about",
            "no radio reception either",
            "never on any radio",
            "at all, biased",
            "xxxxxxxxxxx",
            "don't' listen",
            "do no listen to the radio",
            "dont kno",
            "cant remember title of",
            "t listen on the radio to political programmes",
            "cant recall",
            "dont use radio",
            "i dont listen to the radio",
            "radio also cannot be trusted",
            "i do not have tv",
            "n/a no radio",
            "did not listen",
            "have not watched any",
            "do not listen",
            "none - on annual leave",
            "nonenone",
            "any not to do with politics",
            "don,t know",
            "no program",
            "no programmes",
            "dont listen",
            "only listen to music",
            "do not listen to the radio at all",
            "do not listen to the radio",
            "i don't listen to the radio",
            "i do not listen to the radio",
            "do not listen to radio for politics",
            "na -told you i didn't listen to the radio",
            "listening",
            "don't listen",
            "not listened to any",
            "i dont have a radio",
            "dont know name",
            "dont konnw",
            "dont have radio",
            "don't use radio",
            "i don't listen to the radio",
            "don't jnow",
            "don'tknow",
            "don't listen",
            "shshsss","cyfgvhk","vxbvxf","cygvbkj","cggghghgghhg","kkhhhhhghhh","cighffvhgfhvfgvghgghgu",
            "xxxxxxxxxxx","ghvg","doont knoe","popoiyhfg","jhgngng","dk2","no papers","are you serious",
            "none read the last","i dont read them","don't read them","whatever i catch online","dont read newspaper",
            "nothing worth reading",
            "any newspaper",
            "no third paper",
            "dont read them",
            "none, they are biased",
            "i dont read the politcs pages",
            "unsure, what is on the train",
            "i dont know",
            "none on holiday",
            "dont read news papers",
            "sont read",
            "dont read another",
            "none 2",
            "none havent read any",
            "a laugh",
            "tory smear tatics",
            "answer ffs",
            "defenately not the telegraph",
            "don't read  newspaper",
            "never read",
            "dont buy newspapers",
            "all newspapers are pro-union",
            "newspapers? come on! i want facts not overt lies",
            "newspapers have been bought off",
            "what really happened",
            "not shre",
            "don't remember",
            "don't read papers for politics",
            "there are no scottish papers",
            "no other paper",
            "i don't take any",
            "in the press have been an",
            "xxxxxxxxxxxx",
            "none, don't read newspapers(printed)",
            "do people read newspapers",
            "i have been away",
            "na - told you i haven't read newspapers",
            "do not read newspapers so don't know",
            "none of them",
            "none available on this island",
            "i can think of newspapers but i haven't read any newspapapers in the last week",
            "papers are old school only the brainwashed read the crap anymore",
            "do people actually still read newspapers",
            "xxxxxxxxxxx","xxxxxxxxxxxx","xxxxxxxxxxxxxx",
            "i dont read the news on line",
            "no!e",
            "i have been away",
            "hwrhwrh",
            "do not tend to look at politics stories on internet",
            "i'm not telling you that. mind your own business",
            "whichever party/topic i'm interested in at that time",
            "none specific",
            "none at all",
            "i do not use any web sites to follow politics",
            "dont use internet scources",
            "nonenone",
            "dont read about politics on internet",
            "do not tend to look at political stories on internet",
            "dont kno",
            "cant recall",
            "ghhhjj",
            "dont knwo",
            "likkkkkjkjj",
            "do not use internet",
            "blog i can't remember name of",
            "do not tend to look at politics stories on the internet",
            "don't use",
            "i can't even remember the sites i avoid",
            "i have no interest in party politics",
            "don't have an interest in politics",
            "hh","nome","do","others","need","varied",
            "haven't watched tv","nb","none no tv","i dont watch tv","noyb","have not","need night",
            "programme","kn","really","comment","ba","none live no tv","not the bbc","not watched any","anything",
            "been away on hols","dunno","sure",
             
             
            ]


crap_regex = ["^the "]
replace_dict = dict(zip(crap_list,[np.nan]*len(crap_list)))
to_strip = '.!? \n\t/,>-\\<`′\"\'$;'


In [357]:
# [x for x in df[match(df,"tv|radio|paper|internet",case_sensitive=False).index]\
#      .replace(replace_dict).replace(crap_regex,"",regex=True)\
#      .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
#      .replace(replace_dict).stack().value_counts().index if len(x)<=4]

In [352]:
### Maybe it would be easier if I broke down the mappings by wave?

In [353]:
# df[match(df,"tv|radio|paper|internet",case_sensitive=False).index]\
#     .replace(replace_dict).replace(crap_regex,"",regex=True).stack().value_counts().head(400)[-60:]

                   

# pm? no? 1? 2? non? bt? 3? 4? 5?
# pm is a radio station!

# I think anything that's 1 letter is "woops - I want to get out of here" - except maybe "i" (what about 1/2/3 radio?)

In [474]:
tv_replace_dict = {"news night":"newsnight","questiontime":"question time","bbc 1":"bbc1","bbc 1 news":"bbc1 news","bbcnews":"bbc news",
    "bbc newsnight":"newsnight","party political broadcast":"party political broadcasts","bbc question time":"question time",
    "news at 10":"news at ten","bbc news at 10":"bbc news at ten","newnight":"newsnight",
    "leaders debates":"leaders debate","election debates":"election debate","debates":"debate",
    "daily politics show":"daily politics","andrew marr show":"andrew marr",
    "10 o clock news":"news at ten","10 oclock news":"news at ten","10pm news":"news at ten","ten o'clock news":"news at ten",
    "6 o clock news":"6 o'clock news","6 oclock news":"6 o'clock news","6pm news":"6 o'clock news",
    "six o clock news":"6 o'clock news","six o'clock news":"6 o'clock news",                 
    "news at 6":"6 o'clock news","news at six":"6 o'clock news",
    "aljazeera":"al jazeera",
    "andrew mar":"andrew marr","andrew mar show":"andrew marr","andrew marr on sunday":"andrew marr",
    "andrew marrs":"andrew marr","marr":"andrew marr","marr on sunday":"andrew marr","marr show":"andrew marr",
    "bbc  news":"bbc news","bbc new":"bbc news","bbc newa":"bbc news",
    "bbc 10pm news":"bbc news at ten",
    "bbc 6 o clock news":"bbc news at 6","bbc 6 oclock news":"bbc news at 6","bbc 6 o'clock news":"bbc news at 6",
    "bbc news 6pm":"bbc news at 6","bbc news at six":"bbc news at 6","bbc six o'clock news":"bbc news at 6",
    "bbc news24":"bbc news 24","bbc one":"bbc1","bbc one news":"bbc1 news",
    "big questions":"big question",
    "chanel 4 news":"channel 4 news","channel 4news":"channel 4 news","ch4 news":"channel 4 news",
    "ch 4 news":"channel 4 news","c4 news":"channel 4 news","channel four news":"channel 4 news","channel4 news":"channel 4 news",
    "chanel 4":"channel 4","channel4":"channel 4",
    "channel5":"channel 5",
    "daily politcs":"daily politics","daily politic":"daily politics","daily politic show":"daily politics",
    "daily politics bbc":"daily politics","daily polotics":"daily politics","daily poltics":"daily politics",
    "documentary":"documentaries",
    "euro news":"euronews",
    "good morning":"good morning britain",
    "hignfy":"have i got news for you",
    "it news ":"itn news","its news":"itn news",
    "itv  news": "itv news","itv new":"itv news","itvnews":"itv news",
    "itv news at ten":"itv news at 10",
    "leader debate":"leaders debate","leader debates":"leaders debate","leader's debate":"leaders debate",
    "leaders' debate":"leaders debate","leaders' debates":"leaders debate","leadership debate":"leaders debate",
    "leaders interview":"leader interviews",
    "live debates":"live debate",
    "new":"news",
    "news channel":"news channels",
    "newsight":"newsnight","newsmight":"newsnight",
    "panarama":"panorama","panaroma":"panorama",
    "parliament channel":"parliament",
    "paxman interviews":"paxman",
    "peston":"peston on sunday",
    "pm question time":"pmqs","pm questions":"pmqs","pmq":"pmqs","pmq's":"pmqs",
    "prime minister's questions":"pmqs","prime ministers question time":"pmqs","prime ministers questions":"pmqs",
    "question  time":"question time","question times":"question time","question tine":"question time",
    "question tome":"question time","questions time":"question time","qustion time":"question time",
    "russia today":"rt",
    "scotland2015":"scotland 2015",
    "sky new":"sky news","skynews":"sky news",
    "today programme":"today",
    "tv debates":"debate",
    "election broadcast":"party political broadcasts","election broadcasts":"party political broadcasts",
    "party broadcast":"party political broadcasts","party broadcasts":"party political broadcasts",
    "party election broadcast":"party political broadcasts","party election broadcasts":"party political broadcasts",
    "political broadcast":"party political broadcasts","political broadcasts":"party political broadcasts",
    "political debate":"party political broadcasts","political debates":"party political broadcasts","ppb":"party political broadcasts",
    "party debate":"party debates","bbc news channel":"bbc news",
    "10 o'clock news":"news at ten","tv debate":"debate","live debate":"debate","news bbc":"bbc news",
    "parliament":"bbc parliament","7 leader debate":"7 leaders debate","7 party debate":"7 leaders debate",
    "7 way debate":"7 leaders debate","all party debate":"7 leaders debate","al jazera":"al jazeera",
    "andre marr":"andrew marr","andrew mars":"andrew marr","b.b.c. news":"bbc news","bbc 24 hour news":"bbc news 24",
    "bbc 24 hr news":"bbc news 24","bbc bews":"bbc news","bbc newd":"bbc news",
    "bbc parliament channel":"bbc parliament","bbc debates":"bbc debate","bbc election debate":"bbc debate",
    "bbc news night":"newsnight","bbc parliment":"bbc parliament","bbc politics show":"politics show",
    "bbc news 10pm":"bbc news at ten","bbc 6pm news":"bbc news at 6","bbc news at 6pm":"bbc news at 6",
    "bbc qt":"question time","bbc scottish news":"bbc scotland news","bbc tv":"bbc","bbc tv news":"bbc news",
    "bbc2 newsnight":"newsnight","bbcqt":"question time","bc news":"bbc news","bec news":"bbc news","broadcasts":"broadcast",
    "breakfast tv":"breakfast","c4":"channel 4","c4news":"channel 4 news","ch 4":"channel 4","ch4":"channel 4",
    "corrie":"coronation street","daily  politics":"daily politics","daily/sunday politics":"daily politics",
    "election specials":"election special","france24":"france 24","hardtalk":"hard talk","i tv news":"itv news",
    "htv":"itv","i t v":"itv","itv 1":"itv1","itv election debate":"itv debate","itv leader's debate":"itv leaders debate",
    "itv leaders' debate":"itv leaders debate","leaders interview":"leader interview","nesnight":"newsnight",
    "new night":"newsnight","newd":"news","news (bbc)":"bbc news","news itv":"itv news","news on bbc":"bbc news",
    "news on itv":"itv news","news broadcast":"news","news broadcasts":"news","news channels":"news",
    "news programme":"news","news programmes":"news","news programs":"news","other news":"news","newsnght":"newsnight",
    "opposition leaders debate":"opposition debate","parliament live":"parliament","party leaders debate":"leaders debate",
    "party political":"party political broadcasts","pm's questions":"pmqs","politic show":"politics show",
    "polotics show":"politics show","prime minister questions":"pmqs","q time":"question time","qestion time":"question time",
    "qt":"question time","question time bbc":"question time","queston time":"question time","quetion time":"question time",
    "scottish debate":"scottish leaders debate","sky newa":"sky news","sky news channel":"sky news","televised debates":"debate",
    "tv news":"news","scotland debate":"scotland debates","sky  news":"sky news","sky1":"sky","stv  news":"stv news",
    "sunday polotics":"sunday politics","sunday poltics":"sunday politics","scottish leaders' debate":"scottish leaders debate",
    "sunday politic":"sunday politics","bbc sunday politics":"sunday politics","six oclock news":"6 o'clock news",
    "bbc ten o'clock news":"bbc news at ten","10 o'clock news":"news at ten","10 pm news":"news at ten",
    "bbcnews24":"bbc news 24","news 24":"bbc news 24","news24":"bbc news 24","six oclock news":"6 o'clock news",
    "6 o'clock news bbc":"6 o'clock news","a week in politics":"week in politics","andrew marr sunday":"andrew marr",
    "bbc 2":"bbc2","bbc 24 news":"bbc news 24","bbc daily politics":"daily politics","bbc news 24":"bbc news 24","bbc news at one":"bbc news at 1",
    "bbc24":"bbc news 24","bbs news":"bbc news","calender":"calendar","chanel 5":"channel 5","channel  4 news":"channel 4 news",
    "daily poitics":"daily politics","daily politics bbc2":"daily politics","free speach":"free speech","general news":"news",
    "itv 3":"itv3","itv debates":"itv debate","itv live debate":"itv debate","leader interview":"leader interviews",
    "live debate":"debate","live leaders debate":"leaders debate","news channels":"news","news in general":"news",
    "news reports":"news bulletins","news sky":"sky news","newsnight bbc2":"newsnight","newsnoght":"newsnight",
    "northwest tonight":"north west tonight","party debate":"party debates","political programmes":"politics",                   
    "politics programme":"politics","question":"question time","charlie brooker's weekly wipe":"weekly wipe",
    "daily news":"news","4 news":"channel 4 news","5 news":"channel 5 news","bbc1news":"bbc1 news","bbc new 24":"bbc news 24",
    "bbc 2 newsnight":"newsnight","1st debate":"first debate","2nd debate":"second debate","any news programme":"news",
    "bbc regional news":"local bbc news","news channels":"news","bbc politics":"politics","it news":"itv news",
    "6 pm news":"6 o'clock news","al jazzera":"al jazeera","andrew  marr":"andrew marr","andrew marr programme":"andrew marr",
    "andrew neal":"andrew neil","andrew neill":"andrew neil","any other news":"news","b b c":"bbc","b b c news":"bbc news",
    "bbb news":"bbc news","bbbc news":"bbc news","bbc 24":"bbc news 24",
    "bbc 10 o'clock news": 'bbc news at ten',
    'regional news': 'local news',
    'local bbc news': 'bbc local news',
    'gmb': 'good morning britain',
    'jeremy paxman': 'paxman',
    'main news': 'news',
    'bbc reporting scotland': 'reporting scotland',
    'itv1 news': 'itv news',
    'despatches': 'dispatches',
    'debate on itv': 'itv debate',
    'interviews': 'leader interviews',
    'stv debate': 'stv leaders debate',
    'leaders': 'leaders debate',
    'bbc 10 news': 'bbc news at ten',
    'eu debate': 'eu debates',
    'bbc spotlight': 'spotlight',
    'bbc breakfast tv': 'bbc breakfast',
    'googlebox': 'gogglebox',
    'politics daily': 'daily politics',
    'bbc news wales': 'bbc wales news',
    'scottish debates': 'scotland debates',
    'chanel4': 'channel 4',
    'bbc nees': 'bbc news',
    'question timw': 'question time',
    'bbc 10 o clock news': 'bbc news at ten',
    'bbc wales today': 'wales today',
    'questin time': 'question time',
    'bbc 24hr news': 'bbc news 24',
    'bbc welsh news': 'bbc wales news',
    'parliament today': 'today in parliament',
    'election programmes': 'election',
    'newsnigh': 'newsnight',
    'newsnighr': 'newsnight',
    'channel 4  news': 'channel 4  news',
    'channel4news': 'channel 4 news',
    'daily politica': 'daily politics',
    'have i got new for you': 'have i got news for you',
    'bbc midlands today': 'midlands today',
    'pp broadcast': 'party political broadcasts',  
    "i":"",
    'news at ten itv': 'itv news at 10',
    'daily and sunday politics': 'daily politics',
    "1 o'clock news": 'news at one',
    'bbc this week': 'this week',
    'news at 6pm': 'bbc news at 6',
    'sunday poitics': 'sunday politics',
    'question tiime': 'question time',
    'live election debate': 'election debate',
    'news bulletin': 'news bulletins',
    'ch5': 'channel 5',
    'bbc news at 10pm': 'bbc news at ten',
    'dailypolitics': 'daily politics',
    'channel 4  news': 'channel 4 news',
    'bbc look north': 'look north',
    'election night': 'election tonight',
    'news night scotland': 'newsnight scotland',
    'leaders debate itv': 'itv leaders debate',
    'debate programmes': 'debate',
    'leaders debate stv': 'stv leaders debate',
    'election leaders debate': 'leaders debate',
    'evan davis interviews': 'evan davies',
    'sky newd': 'sky news',
    'documentry': 'documentaries',
    'this week in politics': 'week in politics',
    "leader's debates": 'leaders debate',
    'neswnight': 'newsnight',
    'parliment': 'parliament',
    'bbc lunch time news': 'bbc news at 1',
    "stv debates":"stv leaders debates",
    "parliamentary channel":"bbc parliament",
    "live debate itv":"itv debate",
    "channel 4 leaders debate":"channel 4 debate",
    "live tv debate":"live debate",
    "itn news at 10":"itv news at 10","bbc news programme":"bbc news","2nd leaders debate":"second debate",
    "bbc 2 news":"bbc2 news","sunday/daily politics":"daily politics","question yime":"question time",
    "uk debate":"debate","9 oclock news":"9 o'clock news","newsnigth":"newsnight","alan marr":"andrew marr","nees":"news",
    "news on 4":"channel 4 news","prime minsters questions":"pmqs","dp":"daily politics","debate itv":"itv debate",
    "bbq news":"bbc news","election debate 2":"second debate","bbc local":"bbc local news","bbc local":"bbc local news",
    "daily polatics":"daily politics","debate bbc":"bbc debate","sky news sunrise":"sunrise",                   
    "bbc coverage":"bbc news","news round":"newsround","debate programme":"debate","bbc news bulletins":"bbc news",
    "bbc new channel":"bbc news","bcc news":"bbc news","news at ten bbc":"bbc news at ten","bbc look east":"look east",
    "prime minister question time":"pmqs","televised debate":"debate","it v news":"itv news","sunday politcs":"sunday politics", 
    "pms question time":"pmqs","newsnite":"newsnight","priministers questions":"pmqs", "motd":"match of the day",
    "television news":"news","bbci":"bbc1","bbc six oclock news":"bbc news at 6","quesiton time":"question time",
    "one o'clock news":"news at one","news bbc1":"bbc1 news","primeministers question time":"pmqs","pms questions":"pmqs",
    "sharpend":"sharp end","sky tv":"sky","question tim":"question time","bbc 6o'clock news":"bbc news at 6",                   
    "news program":"news","paper":"newspaper","sky news press preview":"sky press preview","breakfast bbc1":"bbc breakfast news",
    "good morning britian":"good morning britain","breakfast bbc":"bbc breakfast news","bbc breakfast show":"bbc breakfast news",
    'nrws': 'news',
    'day break': 'daybreak',
    'daliy politics': 'daily politics',
    'cameron v milliband': 'cameron vs miliband',
    'kaiser report': 'keiser report',
    'local bbc': 'bbc local news',
    'first leaders debate': 'first debate',
    'leaders debate scotland': 'scottish leaders debate',
    'new at ten': 'news at ten',
    'breakfast on bbc': 'bbc breakfast',
    'news on bbc1': 'bbc1 news',
    'questiom time': 'question time',
    'referendum debates': 'referendum debate',
    'daly politics': 'daily politics',
    'stv leaders debates': 'stv leaders debate',
    'bbc south today': 'south today',
    'itv local': 'itv local news',
    'ten oclock news': 'news at ten',
    'monaghan': 'murnaghan',
    'bbc 6 news': "6 o'clock news",
    'question tme': 'question time',
    'bbc news tv': 'bbc news',
    'part political broadcasts': 'party political broadcasts',
    'bbc scotland leaders debate': 'scottish leaders debate',
    'dayly politics': 'daily politics',
    'bb': 'bbc',
    'andrew neil show': 'andrew neil',
    'bbc nws': 'bbc news',
    'news nite': 'newsnight',
    'local itv news': 'itv local news',
    'polatics show': 'politics show',
    'daily politics debate': 'daily politics',
    'andrew mart': 'andrew marr',
    'pp broadcasts': 'party political broadcasts',
    'andy marr': 'andrew marr',
    'sky 24 hour news': 'sky news 24',
    'press preview (sky news)': 'sky press preview',
    'sky mews': 'sky news 24',
    'radio4': 'radio 4',
    'general news programmes': 'news',
    'aljazera': 'al jazeera',
    'aljezera': 'al jazeera',
    'stv scotland tonight': 'scotland tonight',
    'bbcdp': 'daily politics',
    'bbc 10 pm news': 'bbc news at ten',
    'daily pollitics': 'daily politics',
    'wright show': 'wright stuff',
    'channel 4 news at 7pm': 'channel 4 news at 7',
    'scotlands politics': 'politics scotland',
    'local news bbc': 'bbc local news',
    'bbcnews 24': 'bbc news 24',
    '6.00 news': 'news at 6',
    'andrew mars show': 'andrew marr',
    'uk news': 'news',
    'documentories': 'documentaries',
    'newsnignt': 'newsnight',
    'lunch time news': 'lunchtime news',
    'st news': 'stv news',
    "bbc leaders' debate": 'bbc leaders debate',
    'uk leaders debate': 'leaders debate',
    'charlie brooker': 'weekly wipe',
    'bbc. news': 'bbc news',
    'newsniggt': 'newsnight',
    'this  week': 'this week',
    'any news program': 'news',
    'bbc news 10 pm': 'bbc news at ten',
    'goggle box': 'gogglebox',
    'ch 5 news': 'channel 5 news',
    'channel 4 debates': 'channel 4 debate',
    'cameron/milliband debate': 'cameron vs miliband',
    'b bc news': 'bbc news',
    'andrew mare': 'andrew marr',
    'political programme': 'politics',
    'weekly polotics': 'weekly politics',
    'andy marr show': 'andrew marr',
    'coro': 'coronation street',
    'it. news': 'itv news',
    'bbc 2 daily politics': 'daily politics',
    'bbc 10.00 news': 'bbc news at ten',
    'question timme': 'question time',
    'itn news at ten': 'itv news at 10',
    'national news bbc': 'bbc national news',
    'andrew carr': 'andrew marr',
    'first tv debate': 'first debate',
    'bbc newss': 'bbc news',
    'question timr': 'question time',
    'bbc north west': 'north west tonight',
    'sunday morning bbc': 'sunday morning',
    '1st election debate': 'first debate',
    'cameron/milliband': 'cameron vs miliband',
    'politics sunday': 'sunday politics',
    'television debates': 'debate',
    '5 way debate': '5 leaders debate',
    'debate with 7 leaders': '7 leaders debate',
    'bbc news @ 6': 'bbc news at 6',
    'bbc election tonight': 'election tonight',
    'topgear': 'top gear',
    'newsnigjt': 'newsnight',
    'bbc news scotland': 'bbc scotland news',
    'bbc election': 'election',
    'sunday politics shows': 'sunday politics show',
    'bbc1 news at ten': 'bbc news at ten',
    'bbc news 10': 'bbc news at ten',
    'politics prog': 'politics',
    'al jazeera english': 'al jazeera',
    'channel five news': 'channel 5 news',
    'box news': 'fox news',
    'bbc news chanel': 'bbc news',
    'news on all channels': 'news',
    '6 pm bbc news': 'bbc news at 6',
    'andrew neil politics': 'andrew neil',
    'b&c news': 'bbc news',
    'newyddion 9': 'newyddion',
    'andrew neil interviews': 'andrew neil',
    'sky news debate': 'sky debate',
    'bbc breakfast time': 'bbc breakfast',
    'new 24': '24 hour news',
    'ppbs': 'party political broadcasts',
    'bbc news etc': 'bbc news',
    '10pm news bbc': 'bbc news at ten',
    'breakfast tv bbc': 'bbc breakfast',
    'andrew marrs show': 'andrew marr',
    '5 party debate': '5 leaders debate',
    'evening news bbc': 'bbc evening news',
    'daily poliyics': 'daily politics',
    'east enders': 'eastenders',
    'panaorama': 'panorama',
    "bbc 10 o' clock news": 'bbc news at ten',
    'newstime': 'news time',
    'bac news': 'bbc news',
    'bbc nes': 'bbc news',
    'newswipe': 'weekly wipe',
    "6 0'clock news": 'news at 6',
    'live parliament': 'parliament',
    'sunday politics bbc': 'sunday politics',
    'polotics today': 'politics today',
    'bbc 10 oclock news': 'bbc news at ten',
    'news national': 'national news',
    'bbc newschannel': 'bbc news',
    'itv nes': 'itv news',
    'parliment today': 'today in parliament',
    'question time debate': 'question time',
    'itv newd': 'itv news',
    'live tv debates': 'live debate',
    'question time bbc1': 'question time',
    'eu referendum debate': 'eu debates',
    'right stuff': 'wright stuff',
    'questio time': 'question time',
    "pm's question time": 'pmqs',
    'russia today - rt': 'rt',
    'parliment channel': 'bbc parliament',
    'five news': 'channel 5 news',
    'news week': 'newsweek',
    '3 leaders question time': 'leaders question time',
    'news ight': 'newsnight',
    'bbc two': 'bbc2',
    'political leaders debate': 'leaders debate',
    'bbc news programmes': 'bbc news',
    'daily politics (bbc2)': 'daily politics',
    'farage and cameron': 'cameron and farage',
    '6oclock news': 'news at 6',
    'bbc or itv news': 'bbc/itv news',
    'bbc ews': 'bbc news',
    '3 leaders debate': 'leaders question time',
    'scotland tonight stv': 'scotland tonight',
    'andrew mart show': 'andrew marr',
    "it's news": 'itv news',
    'bbc news 6': 'bbc news at 6',
    'debate on channel 4': 'channel 4 debate',
    "prime minister's question time": 'pmqs',
    'i t v news': 'itv news',
    'question time on bbc': 'question time',
    'party politics': 'party political broadcasts',
    'daily politics debates': 'daily politics',
    'interviews with leaders': 'leaders interviews',
    'chat': 'chat shows',
    'chan 4 news': 'channel 4 news',
    'bbc1 news at 10': 'bbc news at ten',
    "bbc one o'clock news": 'bbc news at 1',
    'second leaders debate': 'second debate',
    'paxman debates': 'paxman',
    'andew marr': 'andrew marr',
    'bbc six o clock news': 'bbc news at 6',
    'leaders election debate': 'leaders debate',
    "10 o'clock bbc news": 'bbc news at ten',
    'tv election debate': 'election debate',
    'news at ten (bbc)': 'bbc news at ten',
    'currant affairs': 'current affairs',
    'paxman interview': 'paxman',
    'david dimbleby': 'dimbleby',
    'international news': 'world news',
    'dispaches': 'dispatches',
    'primeministers questions': 'pmqs',
    "6 o'clock news bbc1": 'bbc news at 6',
    'bbc mews': 'bbc news',
    'miliband v cameron': 'cameron vs miliband',
    'al jeezera': 'al jazeera',
    'party politicals': 'party political broadcasts',
    'election campaign': 'election',
    '4news': 'channel 4 news',
    'wales news': 'welsh news',
    "scotland's politics": 'politics scotland',
    '60clock news': "6 o'clock news",
    'scottish bbc': 'bbc scotland',
    'news bbc or itv': 'bbc/itv news',
    '7 leaders': '7 leaders debate',
    'sky tv news': 'sky news',
    'bec breakfast': 'bbc breakfast',
    'aljezeera': 'al jazeera',
    'obrien': "o'brien",
    'rt uk': 'rt',
    'bb1': 'bbc1',
    '9 o clock news': "9 o'clock news",
    'n ewsnight': 'newsnight',
    'look northwest': 'look north west',
    'scottish leaders debate bbc': 'scottish leaders debate',
    'andrew marrr': 'andrew marr',
    'bbc 6 o clock': 'bbc news at 6',
    'leaders debate on itv': 'itv leaders debate',
    'rt.com': 'rt',
    'politic programme': 'politics',
    'hignify': 'have i got news for you',
    'cameron/miliband': 'cameron vs miliband',
    'jeremy paxman interviews': 'paxman',
    'politics show bbc2': 'politics show',
    'jack dee election helpdesk': 'jack dee',
    'itv sharpend': 'sharp end',
    'scotlamd 2015': 'scotland 2015',
    "bbc news 6 o'clock": 'bbc news at 6',
    'chan 4': 'channel 4',
    'local itv': 'itv local news',
    'television debate': 'debate',
    'bb news': 'bbc news',
    'bbc  news 24': 'bbc news 24',
    'breakfast on bbc 1': 'bbc breakfast',
    'news progs': 'news',
    'news at 1': 'news at one',
    '7 person debate': '7 leaders debate',
    'bbc 4 news': 'bbc4 news',
    'sky debates': 'sky debate',
    't v news': 'news',
    'news stations': 'news',
    'bbc nrews': 'bbc news',
    'live election debates': 'election debate',
    'debate with leaders': 'leaders debate',
    '7-way debate': '7 leaders debate',
    'coranation street': 'coronation street',
    'queation time': 'question time',
    'newanight': 'newsnight',
    'sunday polatics': 'sunday politics',
    'election debate live': 'election debate',
    'newsnigt': 'newsnight',
    't v debate': 'debate',
    'election debate on itv': 'itv debate',
    "nine o'clock news": "9 o'clock news",
    'lunch news': 'lunchtime news',
    'bbc breckfast': 'bbc breakfast',
    "6 o'clock bbc news": "6 o'clock news",
    'national news programmes': 'national news',
    'news bullitins': 'news bulletins',
    'bb1 news': 'bbc1 news',
    'andrewmarr': 'andrew marr',
    'daily politicts': 'daily politics',
    'c4 7pm news': 'channel 4 news at 7',
    'channel 4 newa': 'channel 4 news',
    'andew marr show': 'andrew marr',
    'round the house': 'around the house',
    'leaders debate live': 'leaders debate',
    "challengers' debate": 'challengers debate',
    'itv 7 leaders debate': '7 leaders debate',
    'nerwsnight': 'newsnight',
    'press review': 'sky press review',
    "bbc's news": 'bbc news',
    'newsnight (bbc)': 'newsnight',
    'iyv news': 'itv news',
    'cameron and miliband live': 'cameron vs miliband',
    'stv scottish leaders debate': 'stv leaders debate',
    'reginal news': 'local news',
    'bbcsp': 'sunday politics',
    'polictics show': 'politics show',
    'bbc question times': 'question time',
    'itv 1 news': 'itv news',
    'leadership interview': 'leaders interviews',
    'andrew marr sunday politics': 'andrew marr',
    'good mornimg': 'good morning britain',
    'bbc newx': 'bbc news',
    'election debate 1': 'first debate',
    'evan davies interviews': 'evan davies',
    'politcs scotland': 'politics scotland',
    'bbcbreakfast': 'bbc breakfast',
    'itc news': 'itv news',
    'bbc news national': 'bbc national news',
    'freespeech': 'free speech',
    'sky newspaper review': 'sky press review',
    'electoral debates': 'election debate',
    'daily poiltics': 'daily politics',
    "jack dee's election help desk": 'jack dee',
    'live debate with jeremy paxman': 'paxman',
    'itv central news': 'central news',
    'news on any channel': 'news',
    'bbc any questions': 'any questions',
    'dily politics': 'daily politics',
    'bbc today': 'today',
    'politics today (andrew neil)': 'politics today',
    'debate with paxman': 'paxman',
    'question ime': 'question time',
    '6 news': "6 o'clock news",
    'rt today': 'rt',
    "10 o'clock news bbc": 'bbc news at ten',
    'sp': 'sunday politics',
    'news - bbc': 'bbc news',
    'poltics show': 'politics show',
    'nwes': 'news',
    'daily politcis': 'daily politics',
    'itv/bbc news': 'bbc/itv news',
    'sty news': 'stv news',
    'news (sky)': 'sky news',
    'leaders debate bbc': 'bbc leaders debate',
    'party leader debates': 'leaders debate',
    'party ads': 'party political broadcasts',
    'discussion programmes': 'political discussion',
    'okk': 'ok',
    'john oliver': 'last week tonight',
    'sunday poli': 'sunday politics',
    'sunday morning show': 'sunday morning',
    'bbc breafast': 'bbc breakfast',
    "first minister's questions": 'first ministers questions',
    'itv regional news': 'itv local news',
    'daily polics': 'daily politics',
    "bbc 1 o'clock news": 'bbc news at 1',
    'news chanels': 'news',
    'bbc24 news': 'bbc news 24',
    'challenger debate': 'challengers debate',
    'rt uk news': 'rt news',
    'biased bbc news': 'bbc news',
    'andrew marr shoe': 'andrew marr',
    'tonight show': 'tonight',
    'oneshow': 'one show',
    'daily polices': 'daily politics',
    'leaders televised debate': 'leaders debate',
    'bbcparliament': 'bbc parliament',
    'daily sunday politics': 'daily politics',
    'channal 4 news': 'channel 4 news',
    '6 o`clock news': "6 o'clock news",
    'daily polictics': 'daily politics',
    'dailey politics': 'daily politics',
    'sunday politucs': 'sunday politics',
    'various news channels': 'news',
    'bbc breakfast/news': 'bbc breakfast',
    'interview with party leaders': 'leader interviews',
    'news night bbc2': 'newsnight',
    'aj': 'al jazeera',
    'daily politocs': 'daily politics',
    'channel 4 dispatches': 'dispatches',
    'al jazerra': 'al jazeera',
    'special election programmes': 'election special',
    '5 leader debate': '5 leaders debate',
    'parlament': 'parliament',
    'any documentary': 'documentaries',
    'russia today news': 'rt news',
    'murnagan': 'murnaghan',
    'debates between leaders': 'leaders debate',
    '24 hr news': '24 hour news',
    'election party broadcasts': 'party political broadcasts',
    'paxman in brussels': 'paxman',
    'bbc 2 news night': 'newsnight',
    'itv 10pm news': 'itv news at 10',
    "andrew marr's show": 'andrew marr',
    'bbc 4 world news': 'bbc world news',
    'have i got news 4 u': 'have i got news for you',
    'sky election debate': 'sky debate',
    'bbc 6 pm news': 'bbc news at 6',
    'bbc politcs': 'politics',
    'cnannel 4 news': 'channel 4 news',
    'bbc1  news': 'bbc1 news',
    'bbc 6.00 pm news': 'bbc news at 6',
    'cameron v miliband': 'cameron vs miliband',
    'sky paper review': 'sky press review',
    'leaders  debate': 'leaders debate',
    "ten o'clock news bbc": 'bbc news at ten',
    'it.v news': 'itv news',
    'debated': 'debate',
    'ten o clock news': 'news at ten',
    'marrshow': 'andrew marr',
    'sunday poliitics': 'sunday politics',
    'any debates': 'debate',
    'leaders debate in scotland': 'scottish leaders debate',
    'documentries': 'documentaries',
    'c4  news': 'channel 4 news',
    'itv 10 news': 'itv news at 10',
    '10 o/c news': 'news at ten',
    'big 7 debate': '7 leaders debate',
    'daily politucs': 'daily politics',
    'daily politics (bbc)': 'daily politics',
    'scottish leadrs debate': 'scottish leaders debate',
    'seven leaders debate': '7 leaders debate',
    'a debate': 'debate',
    'general election debate': 'election debate',
    'debate with 5 leaders': '5 leaders debate',
    '6.00 news bbc': 'bbc news at 6',
    'local news programme': 'local news',
    'bbc 1news': 'bbc1 news',
    'news events': 'news',
    'bbc news/debate': 'bbc news',
    'b.b.c.news': 'bbc news',
    'quetiontime': 'question time',
    'andrew marr politics show': 'andrew marr',
    'bbc news at 10.00': 'bbc news at ten',
    'pmqt': 'pmqs',
    'leders debate': 'leaders debate',
    'channel 4 news 7pm': 'channel 4 news at 7',
    'jeremy paxman interview': 'paxman',
    'debate live': 'live debate',
    'nes': 'news',
    'news - all channels': 'news',
    'bbc news at 1pm': 'bbc news at 1',
    'bbd news': 'bbc news',
    'paxman debate': 'paxman',
    'bbc the politics show': 'politics show',
    'sky news paper review': 'sky press review',
    'bews': 'news',
    'scottish first ministers questions': 'first ministers questions',
    'newsnnight': 'newsnight',
    'ch5 news': 'channel 5 news',
    'question time leaders debate': 'leaders question time',
    'pamorama': 'panorama',
    'sunday politicts': 'sunday politics',
    'andrw marr show': 'andrew marr',
    'andrew niel': 'andrew neil',
    'any news channel': 'news',
    'today r4': 'today',
    'various debates': 'debate',
    "leaders' election debate": 'leaders debate',
    'election special bbc': 'election special',
    'andrew married show': 'andrew marr',
    'alan mar show': 'andrew marr',
    "party leaders' debate": 'leaders debate',
    'local new': 'local news',
    'sky papers review': 'sky press review',
    '7 party leaders': '7 leaders debate',
    'news coverage': 'news',
    'paper review': 'sky press review',
    'mewsnight': 'newsnight',
    'andrew made show': 'andrew marr',
    'newsniht': 'newsnight',
    'bbc 3': 'bbc3',
    'bbc panorama': 'panorama',
    'paxman on europe': 'paxman',
    'itv leadership debate': 'itv leaders debate',
    'chanel4 news': 'channel 4 news',
    'other debate': 'debate',
    'news at 10 (bbc)': 'bbc news at ten',
    'questionstime': 'question time',
    'cameron and milliband debate': 'cameron vs miliband',
    'bbc nerws': 'bbc news',
    'rt headline news': 'rt news',
    'ndrew marr': 'andrew marr',
    'aljizera': 'al jazeera',
    'general election debates': 'election debate',
    '5news': 'channel 5 news',
    'bbc questiontime': 'question time',
    'nws': 'news',
    'eastender': 'eastenders',
    'andrew marr interviews': 'andrew marr',
    'quiestion time': 'question time',
    'news @ 10': 'bbc news at ten',
    'cameron/miliband debate': 'cameron vs miliband',
    'c4 news at 7': 'channel 4 news at 7',
    'thenews': 'news',
    'kieser report': 'keiser report',
    'c5 news': 'channel 5 news',
    'scotland to day': 'scotland today',
    "bbc nine o'clock news": 'bbc news at 9',
    'today on radio 4': 'today',
    'bbc nnews': 'bbc news',
    "leaders' question time": 'leaders question time',
    'various news': 'news',
    'q t': 'question time',
    'regular news': 'news',
    'debate2': 'debate',
    'recording of question time': 'question time',
    'bbc1 news at 6': 'bbc news at 6',
    'c5': 'channel 5',
    'daily politics,bbc2': 'daily politics',
    'screenwipe': 'weekly wipe',
    'debate program': 'debate',
    'bic news': 'bbc news',
    'corronation street': 'coronation street',
    'channel four': 'channel 4',
    'thisweek': 'this week',                   
                   
                   
                   
}







vc = df[match(df,"tv",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(tv_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(120)[-60:]
# vc.head(300).sort_index().head(60)[-60:]
vc.head(60)[-60:]
vc





# 1 o'clock news              5
# 24 hour news                6
# 2nd election debate         5
# 2nd leaders debate          4
# 9 o'clock news              5
# 9 oclock news               4

# it news -> prob itv rather than itn but ambiguous!
# leaders -> ambiguous (debate/interview)
# hh,i,

95744 0.8933614639037433 0.9432340407754011


bbc news                           17588
question time                      10718
newsnight                           9671
news                                8918
sky news                            4892
                                   ...  
political program                      1
bbc news at six/wales today            1
bbc question tme                       1
news & follow ups to the debate        1
sky atlantic                           1
Length: 5238, dtype: int64

In [476]:
vc.to_csv('../BES_analysis_data/W19_comb_strings\\tv.csv')

In [488]:
vc.head(400)[-60:]

voteforpolicies.org.uk        3
important internet sources    3
firefox                       3
blog menai                    3
russel brand trews            3
metafilter                    3
reuters news                  3
birmingham mail               3
bbc newa                      3
con home                      3
wordpress                     3
guardianonline                3
sunnation                     3
pornhub                       3
london evening standard       3
edp                           3
pistonheads                   3
independent on line           3
no comment                    3
itv wales                     3
stormfront                    3
global                        3
national collective           3
evening news                  3
channel4.com                  3
government website            3
press and journal             3
hull daily mail               3
nn                            3
express and star              3
bbconline                     3
times re

In [478]:
radio_replace_dict = {"radio 5 live":"5 live","today programme":"today","bbc radio 4":"radio 4","bbc radio 2":"radio 2",
    "five live":"5 live","radio4":"radio 4","radio2":"radio 2","bbc2":"radio 2","radio2":"radio 2","bbc4":"radio 4",
    "5live":"5 live","bbc 2":"radio 2","bbc 4":"radio 4","bbc1":"radio 1","bbc radio 1":"radio 1","bbc 5 live":"5 live",
    "bbc radio":"bbc","bbc world service":"world service","today program":"today","radio news":"news","radio1":"radio 1",
    "r4":"radio 4","radio 4 today":"today","today radio 4":"today","today on radio 4":"today","r4 today":"today","today r4":"today",
    "world at 1":"world at one","bbc radio 2 news":"radio 2 news","bbc radio 4 news":"radio 4 news","radio five live":"5 live",
    "bbc five live":"5 live","radio one":"radio 1","radio 4 today programme":"today","radio four":"radio 4","bbc today":"today",
    "bbc radio 5 live":"5 live","bbc today":"today","r4 news":"radio 4 news","today show":"today","bbc radio 5":"radio 5",
    "pienaars politics":"pienaar's politics","five live drive":"5 live drive","radio two":"radio 2","radio5":"radio 5",
    "6 o'clock news":"six o'clock news","bbc 1":"radio 1","r2":"radio 2","radio five":"radio 5","today (radio 4)":"today",
    "today prog":"today","bbc r2":"radio 2","bbc r4":"radio 4","r4 today programme":"today","6pm news":"six o'clock news",
    "today, radio 4":"today","bbc radio2":"radio 2","radio 5live":"5 live","today (r4)":"today","today - radio 4":"today",
    "bbc r4 today":"today","radio3":"radio 3","today programme radio 4":"today","today on r4":"today","bbc2 news":"radio 2 news",
    "bbc radio two":"radio 2","bbc 5":"radio 5","new":"news","news radio 4":"radio 4 news","fivelive":"5 live",
    "bbc 2 news":"radio 2 news","radio4 news":"radio 4 news","bbc 5live":"5 live","radio4 today":"today",
    "today on radio4":"today","radio5 live":"5 live","bbc radio four":"radio 4","bbc radio 1 news":"radio 1 news",
    "radio four news":"radio 4 news","today radio4":"today","pienars politics":"pienaar's politics",
    "pinaars politics":"pienaar's politics","r5 live":"5 live","radio2 news":"radio 2 news","bbc4 news":"radio 4 news",
    "bbc4 today":"today","bbc5":"radio 5","bbc radio 3":"radio 3","bbc r4 news":"radio 4 news","local":"local radio",
    "bbc scotland":"radio scotland","radio scotland news":"radio scotland","bbc local radio":"local radio",
    "local radio news":"local news","radio 4 pm":"pm","jeremy vine show":"jeremy vine","jeremy vine radio 2":"jeremy vine",
    "bbc radio news":"bbc news","pm radio 4":"pm","talk sport":"talksport","bbc wales":"radio wales",
    "five live breakfast":"5 live breakfast","pm on radio 4":"pm","drive":"5 live drive",
    "heart fm":"heart","capital fm":"capital","p.m":"pm","women's hour":"woman's hour",
    "5live breakfast":"5 live breakfast","radio 2 jeremy vine":"jeremy vine","bbc radio 4 today":"today",
    "pm programme":"pm","radio 1 newsbeat":"newsbeat","r4 pm":"pm","lbc nick ferrari":"nick ferrari",
    "womens hour":"woman's hour","pm r4":"pm","news on radio 4":"radio 4 news","five live news":"5 live news",
    "smooth radio":"smooth","absolute radio":"absolute","bbc radio wales":"radio wales","local bbc":"local radio","bbc local":"local radio",
    "bbc 4 news":"radio 4 news","womans hour":"woman's hour","bbc3":"radio 3","vine":"jeremy vine","bbc radio one":"radio 1",
    "bbc radio4":"radio 4","r5":"radio 5","nick ferrari lbc":"nick ferrari","local bbc radio":"local radio",
    "5live drive":"5 live drive","bbc pm":"pm","bbc newsbeat":"newsbeat","classic":"classic fm","local station":"local radio",
    "jeremy vine r2":"jeremy vine","lbc radio":"lbc","lbc 97.3":"lbc","questiontime":"question time","bbc r4 pm":"pm",
    "local radio station":"local radio","j vine":"jeremy vine","bbc local news":"local news",
    "jeremy vine on radio 2":"jeremy vine","pm - radio 4":"pm","r1":"radio 1","radio 4 any questions":"any questions",
    "pm (radio 4)":"pm","5live news":"5 live news","bbc radio 4 pm":"pm","jermey vine":"jeremy vine",
    "r4 world at one":"world at one","bbc radio solent":"radio solent","news on radio 2":"radio 2 news",
    "local bbc news":"local news","bbc radio 2 jeremy vine":"jeremy vine","bbc radio 4 today programme":"today",
    "ian dale":"lbc iain dale","classical fm":"classic fm","bbc world service news":"world service news",
    "pm at 5pm":"pm","nick ferrari on lbc":"nick ferrari","pinnars politics":"pienaar's politics","p m":"pm",
    "news broadcasts":"news bulletins","pinaar's politics":"pienaar's politics","bbc radio4 news":"radio 4 news",
    "r3":"radio 3","bbc 3":"radio 3","capital radio":"capital","piennars politics":"pienaar's politics",
    "bbc radio1":"radio 1","bbc one":"radio 1","news bbc":"bbc news","bbc radio 5 news":"radio 5 news",
    "radio  4 in general":"radio 4","radio 4 programmes":"radio 4","radio 4 generally":"radio 4",
    "jeremey vine":"jeremy vine","bbc radio 1 newsbeat":"newsbeat","r2 news":"radio 2 news","pm (r4)":"pm",
    "news radio 2":"radio 2 news","jeremy vine bbc radio 2":"jeremy vine","pienar's politics":"pienaar's politics",
    "smooth fm":"smooth","james o'brien lbc":"james o'brien","iain dale lbc":"lbc iain dale","bbc radio five live":"5 live",
    "today programme r4":"today","today programe":"today","pinar's politics":"pienaar's politics",
    "pienar's politics":"pienaar's politics","any question":"any questions","pm, radio 4":"pm",
    "radio5live":"radio 5 live","jeremy vine (radio 2)":"jeremy vine","radio 4 today program":"today",
    "radio 4 today prog":"today","local bbc station":"local radio","deadringers":"dead ringers","radio two news":"radio 2 news",
    "radio 5 live":"5 live","kiss fm":"kiss","news reports":"news bulletins","bbc news bulletins":"news bulletins",
    "radio one news":"radio 1 news","qt":"question time","jeramy vine":"jeremy vine","radio 2 jeremy vine show":"jeremy vine",
    "bb2":"radio 2","ian dale lbc":"lbc iain dale","6 o clock news":"six o'clock news","6 oclock news":"six o'clock news",
    "6 pm news":"six o'clock news","bbc radio scotland":"radio scotland","bbc scotland news":"radio scotland news",
    "bbc radio scotland news":"radio scotland news","chris evans breakfast show":"chris evans","clyde1":"clyde 1",
    "debates":"debate","drivetime":"drive time","file on 4":"file on four","forth 1":"forth one","heart fm news":"heart news",
    "heart radio":"heart","jeremyvine":"jeremy vine","money box":"moneybox","money programme":"moneybox",
    "news bulletin":"news bulletins","news programmes":"news bulletins","other news":"news","pm at 5 pm":"pm",
    "pmq":"pmqs","pmq's":"pmqs","breakfast 5 live":"5 live breakfast","five live daily":"5 live daily",
    "general news":"news","iain dale (lbc)":"lbc iain dale","john humphreys":"john humphries","newa":"news",
    "news bullitins":"news bulletins","6music news":"6 music news","nick ferrari (lbc)":"nick ferrari",
    "question time extra":"question time extra time","radio 4 in general":"radio 4","radio 4 news bulletins":"radio 4 news",
    "radio 4 news at 6":"radio 4 6pm news","radio 4 6 o'clock news":"radio 4 6pm news","radio 4 six o'clock news":"radio 4 6pm news",
    "real radio":"real","six o clock news":"six o'clock news","tfm":"tay fm","today at westminster":"today in westminster",
    "today in parliment":"today in parliament","various radio 4":"radio 4","week at westminster":"week in westminster",
    "westminister hour":"westminster hour","westminster today":"today in westminster","world at one r4":"world at one",
    "today bbc 4":"today","today r 4":"today","you & yours":"you and yours",
    "news at 1":"news at one","one o'clock news":"news at one","1 o'clock news":"news at one","1pm news":"news at one",
    "news at 6":"six o'clock news","news at six":"six o'clock news","ten o'clock news":"news at 10","news at ten":"news at 10",
    "10pm news":"news at 10","10 o'clock news":"news at 10","bbc world news":"world news",
                      
}





vc = df[match(df,"radio",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(radio_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(300).sort_index().head(300)[-60:]
vc.head(170)[-60:]


# radio 1-5 (news)
# bbc (news|london)
# local (radio|news)
# classic fm (news)|heart (news)
# news|news bulletins
# pm|lbc|today|jeremy vine|news quiz|world service|capital|smooth|talksport|pienaar's politics|newsbeat

# 5 live (drive|breakfast|news)
# world at one|any questions|any answers|question time|westminster hour|today in parliament|week in westminster
# radio (scotland|wales)
# good morning (scotland|wales)|world tonight|broadcasting house|woman's hour|magic|kiss|wato|gms


# radio 4 morning/breakfast/morning news???
            
# 5 o'clock news               9
# 6 o'clock news radio 4       5
#                   4

# am, fm -> useful??
# free radio,hourly news,world news,national news
# radio clyde


37910 0.8037457135320496 0.8680031653917172


talk radio                    15
politics                      14
chris evans                   14
scottish news                 14
stephen nolan                 14
real                          14
this morning                  14
file on four                  13
metro                         13
politics today                13
radio devon                   13
gold                          13
radio wales news              13
radio 4 am                    13
radio sheffield               12
radio kent                    12
scotland today                12
good morning                  12
morning call                  12
daily politics                12
radio 4 morning news          12
radio wm                      12
6 music news                  11
radio newcastle               11
taro'r post                   11
nicky campbell                11
radio humberside              11
morning news                  11
wave                          11
week in politics              10
start the 

In [479]:
vc.to_csv('../BES_analysis_data/W19_comb_strings\\radio.csv')

In [480]:
paper_replace_dict = {"mail":"daily mail","daily telegraph":"telegraph","daily mirror":"mirror","daily express":"express",
    "independant":"independent","financial times":"ft","sunday mail":"mail on sunday","standard":"evening standard",
    "local newspaper":"local paper","record":"daily record","daily star":"star","gaurdian":"guardian","dailymail":"daily mail",
    "press & journal":"press and journal","i paper":"i","d telegraph":"telegraph","dail mail":"daily mail",
    "gardian":"guardian","mirrow":"mirror","gurdian":"guardian","telegragh":"telegraph","mos":"mail on sunday",
    "indipendent":"independent","ipaper":"i","daly mail":"daily mail","huffpost":"huffington post",
    "huffington":"huffington post","local press":"local paper","p&j":"press and journal","i newspaper":"i",
    "mail on line":"mail online","d mail":"daily mail","huff post":"huffington post","indy":"independent",
    "saturday mail":"mail on saturday","grauniad":"guardian","daily  mail":"daily mail","guradian":"guardian",
    "dt":"telegraph","telegrapgh":"telegraph","london standard":"evening standard","torygraph":"telegraph",
    "indie":"independent","dailymirror":"mirror","mailonline":"mail online","guadian":"guardian","local news":"local paper",
    "thesun":"sun","local daily":"local paper","es":"evening standard","guardain":"guardian",
    "local papers":"local paper","indpendent":"independent","sunday sun":"sun on sunday","men":"manchester evening news",
    "s times":"sunday times","bbc website":"bbc online","express & star":"express and star","local newspapers":"local paper",
    "indi":"independent","guardian weekly":"guardian","bbc news online":"bbc online","exspress":"express","sun times":"sunday times",
    "daily  mirror":"mirror","local news paper":"local paper","daily maik":"daily mail","telagraph":"telegraph",
    "d.mail":"daily mail","local weekly paper":"local paper","d. telegraph":"telegraph","dm":"daily mail",
    "indepedent":"independent","telegrah":"telegraph","local rag":"local paper","d mirror":"mirror","indipendant":"independent",
    "huffington post uk":"huffington post","daily mirrow":"mirror","mirroe":"mirror","daily telegrapgh":"telegraph",
    "indepentant":"independent","daily mail online":"mail online","telegrath":"telegraph","local daily paper":"local paper",
    "telegaph":"telegraph","tines":"times","observor":"observer","huffingdon post":"huffington post",
    "guaridan":"guardian","dailey express":"express","timed":"times","the sun":"sun","local weekly":"local paper",
    "e standard":"evening standard","indepenent":"independent","independence":"independent","dailly mail":"daily mail",
    "guardiam":"guardian","sundaytimes":"sundaytimes","daily miror":"mirror","theguardian":"guardian","gusrdian":"guardian",
    "daliy mail":"daily mail","daiy mail":"daily mail","sunday time":"sunday times","mirrror":"mirror","daiky mail":"daily mail",
    "huffing ton post":"huffington post","idependent":"independent","guardin":"guardian",
    "daily mail on line":"mail online","daily morror":"mirror","times on sunday":"sunday times","f t":"ft",
    "ns":"new statesman","sum":"sun","son":"sun","daiy telegraph":"telegraph","miror":"mirror","dailytelegraph":"telegraph",
    "p & j":"press and journal","daily times":"times","f.t":"ft","mirrir":"mirror","dail mirror":"mirror",
    "daily fail":"daily mail","telgraph":"telegraph","local guardian":"guardian","daily mai":"daily mail",
    "regional paper":"local paper","dailystar":"star","bbc news website":"bbc news online","daliy mirror":"mirror",
    "m.e.n":"manchester evening news","inderpendant":"independent","p and j":"press and journal",
    "d. mail":"daily mail","daily telegragh":"telegraph","inependent":"independent","giuardian":"guardian","morrir":"mirror",
    "telegrpah":"telegraph","huffpo":"huffington post","d tel":"telegraph","sat times":"saturday times","i daily":"i",
    "evening stardard":"evening standard","daily mirrpr":"mirror","daily rxpress":"express","telegraphy":"telegraph",
    "daily telrgraph":"telegraph","daily maul":"daily mail","local daily newspaper":"local paper","tmes":"times",
    "telegrap[h":"telegraph","daily teleghraph":"telegraph","daily  express":"express","guardians":"guardian",
    "evening std":"evening standard","evening stamdard":"evening standard","local area paper":"local paper",
    "dailyexpress":"daily express","telegrph":"telegraph","mrror":"mirror","daily mail;":"daily mail",
    "dailt mail":"daily mail","aun":"sun","saturday daily mail":"mail on saturday","thetimes":"times","un":"sun",
    "daily mail on sunday":"mail on sunday","the i":"i","indepentdent":"independent","ii":"i","telegrap":"telegraph",
    "i dependant":"independent","huffinton post":"huffington post","guarduian":"guardian","indepenant":"independent",
    "dail;y mail":"daily mail","timea":"times","telrgraph":"telegraph","evening  standard":"evening standard",
    "dsily mail":"daily mail","dail telegraph":"telegraph","guardia":"guardian","daily mirrior":"mirror",
    "daily  telegraph":"telegraph","press& journal":"press and journal","oberver":"observer","dailey mirror":"mirror",
    "telegrapth":"telegraph","localpaper":"local paper","mirro":"mirror","daily mial":"daily mail","insependent":"independent",
    "sunday mirrow":"sunday mirror","new stateman":"new statesman","daily express":"express","daily t":"telegraph",
    "telepgraph":"telegraph","f times":"ft","imes":"times","aberdeen press & journal":"press and journal",
    "aberdeen press and journal":"press and journal","courier and advertiser":"courier","couriet":"courier",
    "dailt telegraph":"telegraph","daily express":"express","daily maily":"daily mail","daily recored":"daily record",
    "eastern daily press":"edp","edp24":"edp","eveningstandard":"evening standard","guardian.co.uk":"guardian online",
    "guardion":"guardian","glasgow hearld":"herald","glasgow herald":"herald","hearld":"herald","herald (glasgow)":"herald",
    "herald/sunday herald":"herald","herlad":"herald","herold":"herald","i/independent":"independent/i",
    "leics mercury":"leicester mercury","lep":"lancashire evening post","mail on sunda":"mail on sunday",
    "mail/mail on sunday":"daily mail","mirtor":"mirror","nation":"national","nationa":"national",
    "national (scotland)":"national","ny times":"new york times","press":"press and journal","s telegraph":"telegraph",
    "sat telegraph":"saturday telegraph","s. times":"times","scotland in sunday":"scotland on sunday",
    "yep":"yorkshire evening post","tlegraph":"telegraph",
    "daily  record":"daily record","daily mirroe":"mirror","dmail":"daily mail","e.d.p":"edp",
    "eadt":"east anglian daily times","east anglian":"east anglian daily times","fh":"falkirk herald",
    "fife press":"fife free press","independent.co.uk":"independent online","local":"local paper",
    "local evening paper":"local paper","local evening post":"local paper","local free paper":"local paper",
    "local gazette":"local paper","local to my area":"local paper","mai":"daily mail","mail  on sunday":"mail on sunday",
    "mi rror":"mirror","msil":"daily mail","new shopper":"news shopper","obseerver":"observer","sun  y":"sun",
    "scotland on sundat":"scotland on sunday","sundaytimes":"sunday times","western morning ~news":"western morning news",
    "(glasgow) herald":"herald","bournemouth daily echo":"bournemouth echo","daily mail/mail on sunday":"daily mail",

                      
                      
}
vc = df[match(df,"paper",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(paper_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(300).sort_index().head(60)[-60:]
vc.to_csv('../BES_analysis_data/W19_comb_strings\\paper.csv')
vc.head(180)[-60:]
# .head(300)[-60:]

# ny times                    5
# 
# mostly blogs???

# guardian/the observer          2
# broadsheets


55306 0.9575091310165262 0.9773080678407406


sport                     5
sunday                    5
sunday independent        5
buzzfeed                  5
google news               5
derby telegraph           5
sky                       5
argus                     5
oxford mail               5
ipswich star              5
york press                5
camden new journal        5
daily sport               5
bolton news               4
herald express            4
fife free press           4
post                      4
ross-shire journal        4
lichfield mercury         4
falkirk herald            4
golwg                     4
worcester news            4
evening telegraph         4
times on saturday         4
evening gazette           4
dandy                     4
scottish express          4
southern reporter         4
today                     4
daily echo                4
independent online        4
local echo                4
bournemouth echo          4
glasgow evening times     3
saturday mirror           3
iq                  

In [481]:
internet_replace_dict = {"bbc.co.uk":"bbc","mail":"daily mail","bbc website":"bbc","mail online":"daily mail",
    "daily telegraph":"telegraph","bbc news website":"bbc news","mail on line":"daily mail","you gov":"yougov",
    "daily mirror":"mirror","guardian.co.uk":"guardian","bbc online":"bbc","huff post":"huffington post",
    "bbc.co.uk/news":"bbc news","mailonline":"daily mail","huffington":"huffington post","daily express":"express",
    "bbc news online":"bbc news","telegraph.co.uk":"telegraph","bbcnews":"bbc news","guardian website":"guardian",
    "bbc news app":"bbc news","dailymail":"daily mail","huffpost":"huffington post","huffing ton post":"huffington post",
    "financial times":"ft","daily mail online":"daily mail","telegraph online":"telegraph","www.bbc.co.uk":"bbc",
    "guardian.com":"guardian","labourlist":"labour list","gaurdian":"guardian","ft.com":"ft",
    "dailymail.co.uk":"daily mail","news.bbc.co.uk":"bbc news","bbc.com":"bbc","theguardian.com":"guardian",
    "russia today":"rt","face book":"facebook","rt.com":"rt","politicalbetting.com":"politicalbetting",
    "bellacaledonia":"bella caledonia","independent.co.uk":"independent","may2015.com":"may2015",
    "independent online":"independent","sky.com":"sky","you tube":"youtube","bbc news site":"bbc news",
    "bec news":"bbc news","telegraph website":"telegraph","bbcnews.co.uk":"bbc news","independent website":"independent",
    "huffington post uk":"huffington post","daily mail on line":"daily mail","daily mail website":"daily mail",
    "local newspaper":"local paper","bbc/news":"bbc news","msm":"msn","www.dailymail.co.uk":"daily mail","msn.com":"msn",
    "thetimes.co.uk":"times","www.telegraph.co.uk":"telegraph","bbc new":"bbc news","timesonline":"times",
    "huffingdon post":"huffington post","sky news app":"sky news","huffinton post":"huffington post",
    "mirror.co.uk":"mirror","coffee house":"spectator coffee house","www.guardian.co.uk":"guardian","bbc app":"bbc",
    "guardian news":"guardian","newstatesman":"new statesman","gardian":"guardian","theguardian.co.uk":"guardian",
    "goggle":"google","itv.com":"itv","facebook links":"facebook","telegraph on line":"telegraph",
    "bella caladonia":"bella caledonia","bbc homepage":"bbc","dailymail.com":"daily mail","huff":"huffington post",
    "dt":"telegraph","mirror on line":"mirror","indy":"independent","ft online":"ft","newstatesman.com":"new statesman",
    "guardian on line":"guardian","mirror online":"mirror","times on line":"times","utube":"youtube",
    "daly mail":"daily mail","u gov":"yougov","bbc web page":"bbc","huffingtonpost.co.uk":"huffington post",
    "guardian app":"guardian","times website":"times","indipendant":"independent","bbc.co,uk":"bbc",
    "express.co.uk":"express","bbc news on line":"bbc news","sun.co.uk":"sun","bbc bews":"bbc news","telegrapth":"telegraph",
    "gurdian":"guardian","bbcnews.com":"bbc news","bbc news channel":"bbc news","guadian":"guardian",
    "bbc home":"bbc","aljazeera":"al jazeera","bbc 1":"bbc1","bbc news.co.uk":"bbc news","bbc on line":"bbc",
    "bbc web site":"bbc","bt home page":"bt","bt internet":"bt","bt.com":"bt","buzz feed":"buzzfeed","commonspace":"common space",
    "conhome":"conservative home","conservative":"conservative home","conservativehome":"conservative home",
    "conservativehome.com":"conservative home","dail mail":"daily mail","email":"emails","fb":"facebook",
    "googlenews":"google news","flip board":"flipboard","golwg360":"golwg 360","government":"gov.uk",
    "green":"green party","green party website":"green party","guardian":"guardian","guido":"guido fawkes",
    "guido fawkes blog":"guido fawkes","herald online":"herald","huffingtonpost":"huffington post",
    "huffpo":"huffington post","independant":"independent","independent newspaper":"independent",
    "liberal democrat voice":"lib dem voice","libdemvoice":"lib dem voice","lord ashcroft polls":"lord ashcroft",
    "may 2015":"may2015","moneysavingexpert":"money saving expert","mse":"money saving expert","msn uk":"msn",
    "msn.co.uk":"msn","new statesman website":"new statesman","newsnet.scot":"newsnet scotland",
    "newsnetscotland":"newsnet scotland","news net scotland":"newsnet scotland","news sites":"news","news websites":"news",
    "ny times":"new york times","order-order":"order order","order-order.com":"order order",
    "political betting.com":"political betting","politicalbetting":"political betting","politics uk":"politics.co.uk",
    "politicshome":"politics home","scotland goes pop":"scot goes pop","scotsman.com":"scotsman","sk":"sky",
    "sky new":"sky news","sky news online":"sky news","sky news website":"sky news","skynews":"sky news",
    "redbox":"red box","reddit.com":"reddit","skynews.co.uk":"sky news","skynews.com":"sky news","snp website":"snp",
    "snp.org":"snp","stv.tv":"stv","sun online":"sun","talk talk":"talktalk","telegrah":"telegraph","telegraph.com":"telegraph",
    "times online":"times","times.co.uk":"times","twitter.com":"twitter","uk polling":"uk polling report",
    "ukip website":"ukip","ukpolling report":"uk polling report","ukpollingreport":"uk polling report",
    "polling report":"uk polling report","virgin media":"virgin","virginmedia":"virgin","wales on line":"wales online",
    "walesonline":"wales online","weegingerdug":"wee ginger dug","wings":"wings over scotland",
    "wingsoverscotland":"wings over scotland","wingsoverscotland.com":"wings over scotland","www.bbc.co.uk/news":"bbc news",
    "yahoo home page":"yahoo","yahoo.co.uk":"yahoo","yahoo.com":"yahoo","yougov.co.uk":"yougov","yougov.com":"yougov",
    "zero hedge":"zerohedge","wos":"wings over scotland","ukpr":"uk polling report","38degrees":"38 degrees",
    "aol.com":"aol","aav":"another angry voice","bbc  news":"bbc news","bbc i player":"bbc iplayer","bbc newd":"bbc news",
    "bbc news 24":"bbc news","bbc news web site":"bbc news","bbc online news":"bbc news","bbc uk":"bbc","bbc,co.uk":"bbc",
    "bbd news":"bbc news","bbs news":"bbc news","bells caledonia":"bella caledonia","brietbart":"breitbart",
    "bt homepage":"bt","btyahoo":"bt yahoo","c4":"channel 4","channel4":"channel 4","c4 news":"channel 4 news",
    "conservative party website":"conservative party","conservatives":"conservative party","conservative website":"conservative party",
    "digitalspy":"digital spy","economist.com":"economist","election forecast":"electionforecast.co.uk",
    "electoralcalculus.co.uk":"electoral calculus","eureferendum":"eu referendum","facebook groups":"facebook",
    "facebook.com":"facebook","france24":"france 24","gogle":"google","google search":"google","google.com":"google",
    "greens":"green party","guidofawkes":"guido fawkes","herald website":"herald","heraldscotland.com":"herald scotland",
    "independant.co.uk":"independent","itv.co.uk":"itv","labour party website":"labour party","labour website":"labour party",
    "libdem voice":"lib dem voice","mail plus":"daily mail","mail.co.uk":"daily mail","news feeds":"news",
    "political  betting":"political betting","politicalbetting":"political betting","reuter":"reuters","reuters.co.uk":"reuters",
    "sky.co.uk":"sky","telegragh":"telegraph","telegraph app":"telegraph","telegraph blogs":"telegraph","thesun":"sun",
    "thisismoney":"this is money","uk gov":"gov.uk","whatscotlandthinks":"what scotland thinks","www.snp.org":"snp",
    "spectator coffeehouse":"spectator coffee house","www.bbcnews":"bbc news","www.ukpollingreport.co.uk":"uk polling report",
    "www.yougov.com":"yougov","yarhoo":"yahoo","you guv":"yougov","youyube":"youtube","guardian online":"guardian",
                         
                         
                         
                         
                         
}
vc = df[match(df,"internet",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(internet_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
vc.to_csv('../BES_analysis_data/W19_comb_strings\\internet.csv')
# vc.head(300).sort_index().head(300)[-60:]
vc.head(170)[-60:]
# .head(300)[-60:]



# party website                  6
# party websites                32
# political party websites       4
# individual party websites       5  
# party website                6
# party websites              32
# political party websites       4

# daily telegraph website
#five thirty eight             4 
# general                         3
# general browsing                3
# local                        9
# local news                   9
# local paper                 28
# newspapers                   9
# various blogs

# other- alternative,rather not say

50636 0.8759380677778655 0.9266529741685757


virgin                     17
vox political              16
spiked                     16
france 24                  15
sunday herald              15
emails                     15
digital spy                14
bbc scotland               14
itn                        14
left foot forward          14
common weal                14
golwg 360                  14
bt yahoo                   13
euronews                   13
canary                     13
bbc iplayer                13
mumsnet                    13
full fact                  12
glasgow herald             12
capx                       12
lord ashcroft              12
washington post            12
commentator                11
orange                     11
zerohedge                  11
week                       11
daily wales                11
daily mash                 10
le monde                   10
eu referendum              10
daily post                 10
electionforecast.co.uk     10
lbc                        10
local news

In [482]:
# df[match(df,"tv|radio|paper|internet",case_sensitive=False).index].replace(replace_dict)\
#     .replace(crap_regex,"",regex=True)\
#     .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
#     .replace(replace_dict).stack().value_counts().head(60)[-60:]




In [483]:
%%time
tvradiopaperinternet = pd.DataFrame()

tv_cols = match(df,"tv",case_sensitive=False).index
tvradiopaperinternet[tv_cols] = df[tv_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(tv_replace_dict).replace("",np.nan).astype('category')

radio_cols = match(df,"radio",case_sensitive=False).index
tvradiopaperinternet[radio_cols] = df[radio_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(radio_replace_dict).replace("",np.nan).astype('category')

paper_cols = match(df,"paper",case_sensitive=False).index
tvradiopaperinternet[paper_cols] = df[paper_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(paper_replace_dict).replace("",np.nan).astype('category')

internet_cols = match(df,"internet",case_sensitive=False).index
tvradiopaperinternet[internet_cols] = df[internet_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(internet_replace_dict).replace("",np.nan).astype('category')

Wall time: 4min 10s


In [484]:
tvradiopaperinternet[tvradiopaperinternet.columns[0]].value_counts()

bbc news                     5891
news                         3456
question time                2370
newsnight                    2134
daily politics               1075
                             ... 
news at 10 bbc                  1
news at 10 (itv)                1
news any channel                1
news 24/7                       1
(only foreign programmes)       1
Name: tv1_1W4, Length: 755, dtype: int64

In [485]:
# What are the most important (TV programmes you watch|Radio programmes you listen to|newspapers (not including online) you read
#               |Internet sources (including online newspapers) you read) for information about politics and the election?
# List up to three programmes or click next if there are no programmes you can think of

tvradiopaperinternet.to_pickle('../BES_analysis_data/W19_comb_strings\\tvradiopaperinternet.zip', compression='zip')
tvradiopaperinternet.to_csv('../BES_analysis_data/W19_comb_strings\\tvradiopaperinternet.csv')

In [450]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [451]:
examples = list(tvradiopaperinternet[tv_cols].stack().value_counts().index)

In [458]:
min_case = 0
max_case = 288

choices = {x:examples[x] for x in range(len(examples[min_case:max_case]))}
# choices


In [459]:
# ex_no=312
fuzz_dict = []
for ex_no in range(288,len(examples)):
    print(examples[ex_no])
    fuzz_dict.append( process.extract(examples[ex_no],choices,
                                scorer=fuzz.token_sort_ratio,
                                processor= lambda x: fuzz.utils.full_process(x,force_ascii=True)) )

sputnik
sky leaders debate
cameron vs miliband
stv local news
nrws
around the house
adventure time
welsh leaders debate
parliament tv
day break
daliy politics
am pm
cameron v milliband
films
radio 2
scottish questions
sunday
itv4
budget
daytime tv
stv news at six
kaiser report
local bbc
central lobby
first leaders debate
radio 2 news
leaders debate scotland
talk shows
bbc election special
online
general
itv wales news
sunday show
head to head debate
meridian
about anglia
wildlife
new at ten
breakfast on bbc
itv news at 6
news on bbc1
questiom time
msn
lunchtime politics
representing border
referendum debates
sky 1
bbc/sky news
yes minister
discussion
chat show
bbc election coverage
nne
htv news
daly politics
teletext
countryfile
s4c news
stv leaders debates
bbc south today
itv local
ten oclock news
westminster
westminster hour
nightly news
monaghan
five live
bill
bbc 6 news
question tme
others
questions
bbc news tv
part political broadcasts
bbc scotland leaders debate
dayly politics
bb

bbc 6 pm news
bbc politcs
judge judy
cnannel 4 news
hhh
celeb juice
evening tv news
bbc1  news
bbc 6.00 pm news
cameron v miliband
sky paper review
leaders  debate
ten o'clock news bbc
it.v news
debated
ten o clock news
bbc scottish debate
marrshow
sunday poliitics
ukip party political broadcast
lab
am
any debates
leaders debate in scotland
welsh debate
documentries
c4  news
itv 10 news
yahoo news
watched
10 o/c news
big 7 debate
gg
week that was
daily politucs
daily politics (bbc)
scottish leadrs debate
seven leaders debate
sony
ff
itv election
one after question time
channel 4/sky debate
poldark
a debate
points west news
general election debate
debate with 5 leaders
cg
itv scottish leaders debate
6.00 news bbc
local news programme
bbc 1news
anything
news events
anglia
bbc news/debate
b.b.c.news
channel 4 evening news
political programs
bbc scotland debate
an la
daily news broadcasts
quetiontime
bbc london
ytv
film 4
welsh assembly
fmqt
teletubbies
andrew marr politics show
bbc east m

election 2015 programmes
10 0clock news bbc
scottish leader's debate
party policital broadcasts
question time the debate
sunday polititics
cameron
profiles of leaders
channel 4 ,news
election live debate
special broadcast on elections
itv re next debate
bbc the great debate
10pm bbc news
hogans hearos
post deabte
labour/con debate
bc newa 24
scotland this week
politics with andrew neil
channel2
panel
daily p[olitics
bedate
today radio 4
news1
any new show
bc2
genaral news
tonight itv
electio broadcast
first
news at lunch
election  broadcast
24 hours news
star wars
scottish debate on bbc
1pm news, bbc1
melissa and joey
sunday lunch
alan carr
local tv station
southeast today
hollypokas
political  debate
sunday ppolitic
jeremey bloody vine
tom bradbury
ky news
david vs miliband
election debate on sky
bt sports
debates when they happen
al jeezera uk news
election tv deabate
scotland tonite
sky news live
itv debate and
algecira  news
cameron & milliband interviews on ch4 and sky news
parlim

i never said i watched live tv, just saw stuff cast onto the tv from the internet. have i got news for you bbc iplayer
scottish tv debates
quest time
bbc new 6 o'clock
2nd multi-party broadcast
leaders debat
hvhh
sunday morning andrew marr one
ed verses cam paxman
leaders dedbate
leaderrs debate
labour
politics bbc2
poitics today
tonight's debate
9o"clock news
bbc election debate 2015
bbc news (6pm)
andrew neil george osborne
morning t.v
andrew marr sshsow
it's debate
bbc morning television
hollyokes
interviews with candidates but i can't remember what the programme was called
scottish political debate
leadership debate scotland
6 oclock
leader interviews (bbc)
daily politics/sunday politics
last leg: election special
pollitics svotland
bbc late evening news
politics europe
new at 6 bbc
bbc general news programmes
pjs questions
dad's army
bbc reginal news
tom bradbys the agenda
managhan
cch4. news
police camera, action
6 o' clock news
bbd news 6pm
quesion time
pledge
i tv
politic show 

part political broadcast
newsnight question time
evening news sky
sk
ross kemp extreme world
evan davies interviewing  nick clegg
leader interviews: ed miliband
panorama - bbc 1 monday
tom bradbury programmes
general election debate- 7 party leaders
afternoon political slot
bbc scotland sunday politics show
news (from a range of channels)
victoria bbc one
political broadcast for snp
itv new s
andrewe mar show
debate on itv on 2ndapril
pm question
24 hour live news
rick stein
nnewsnight scotland
comedy roadshow
2 o'r bae
bbc2 debate show
bbc news lunch time
deal or no dear
westcountry news
six o'clock news bbc
examiner
bbc nres
tv dbate ed milliband had with russel brand
debate, farage, cameron
politics with andrew marr
this week/questiontime
7 o'clock news
election program
th
debatevshow
dave v boris
i.t.v.debate
nws bbc
cameron & miliband live
bbc breakfast programme
campaign reporting/bbc news
one o clock news bbc one
hace i got news for you
lib
cameron and farage live itv
sky news -

...

questen time
gnus on 4
b&b breakfast
yeastenders
rejutedxzxbrsz
question time election leaders special
crap
electtion broadcast
andrew neil's daily politics
rt - uk news
newsnught
bbc news reality check / political news items
internet newspapers
coalition drama
ten-o-clock news
itv agenda
bbc 2 12 noon
election debate 30/04
bbc 1 news morning
bbc - maher
tba
bbc local tv
live debate to be shown 7/4/15
hard talks
any question
bbc interview with nigel farage
who will win the election
tv leader debates
victoria derbishire
wales today bbc
question time (bbc)
bbc analysis of debate
party politicall broadscastspol
channel 4 debate with cameron
press view sky news
that's it
ewa
general election party debate
going underground rt
gams
2cnd leaders debate
paxman's interviews
breakfast news bbc
debates, if they happen
itv daily news
bbc onr
itn newa
party political debates
newsbbc1
regional news -bbc
politics on sundays  a sunday
itv breackfast
newsnight - bbc 2 most nights
cnn all programs
maybe

leadersdebate stv
interview with the leaders
week in review (sky news)
individual leader questioning programme
last nights debate itv
leaders debate (2nd part)
bbc 1  party broadcast
news  in usa
news channel5
question tjme
bbc news rolling coverage
channel 4: battle for downing street
priminister questions time
bbc n
football league show
andrew marr on sunday morning
scots leader debate itv
rt tv
question time§
magic roundabout
campaign highlights
boris v dave
interview with leaders last week
sunday political programmes
channel four news at 7pm
emmerdeale
skky news
leaders interview with the smaller parties
election debate channel 4
leaders debate (7 leaders)
dispatchers
good morning brittan
sbsb
related programmes
scottish debates stv&bbc
ukip
polatics today
6pm news. bbc
party interviews
qhestion time
floating vote london programme
itv scottish
bbc local scottish news
regional news program
channel 4 ne
andrew neils' programme
documenties
daily news programme
safe house
comedy shows


ssu
graham norton
banished
multi party debate
onlysawtwo
anrdew marr
6 oclock news bbc1
news bulletines
party political talks
murnahan sky news
all of the debate
bbc wales 6.30
election sunday
various news chanels
bbc nw
bbt
andrew murs
banning hunting
bbc parliement
itv bews
bbc news / andrew neil show
marr on sunday am
sky news thursday
leaders interviews (bbc)
daily politrics
itv5
national  news
bbc world news today
world news bbc
daily |politics
bbc emt
jeremy paxman on sovereignty
itv news1830
live politial debate
americian dad
scottish leaders debate stv
bu
scottish television news
newsnight on bbc2
battle for 10
breakfast time bbc
bbc news and bbc south
debates on bbc
skynews sunrise
leaders 3 way debate
week in  politics
news local wales
morning show bbc
bbc scotland election
qquestion timw
leaders debate (7)
newsnight  -  bbc
eu debate with cameron and farage
havi i got news for you
bbc news @ 6p.m
tv interviewing of all party leaders
bbc five leaders
bbc night q&a
channl4
pol

first debate election broadcast
bbc news @ 10
battle four number ten
any main news prog
itv news west country
bbc 1 10 o'clock news
time team
prime.ministers questions
bbc television news
news at 1.00
lost
bbc policics show
fly on the wall' documentaries
leader;s debate
itv 7-way debate
david cameron and ed milliband debate with jeremy paxman
bbc young people's debate
bbc itv ch 4 news
we're heart is
am news
saturday
last week's 2 hr debate with 6 mp's inc boris
bbc news 6:00
channel 4 jeremy paxman
tv evening news
bbc 1 news - daily
peston's politics
party political broadcast for labour
wales today news
fm questions
andrew marr (to hear the ridiculous nonsense that they think no-one will check)
any broadcast from westminster
bbc breqkfast
general debate
tyne tees local news
leaders, tv programme
leader discussions
bbc national news 1pm & 6pm
bbc 4 world need
brexit movie
bbc news @ 22.00
itv news 24
live updaste
election debate bbc1
political programs in general
party ploitical broadc

local channel
cam/farage debate
bbc the daily politics
channal4
3rd leader debate
interview with leaders- evan davis
itv 10 pm news
scotand today
sunday bbc morning
bbc-1 news
fhhvg
sky with cameron
bbc beakfast
bbc 10 on clock news
debate bbc1
election programmes if i know about then
cammaron& farrage debate
newesnight
5live
local news station
special events
itn evening news
parliament 131
question time (part)
politics today (bbc)
bb c news
pan aroma
bbc nws 24
scottish question
andrew marr bbc
leaflets
channel  5 news
first debate with 7 leaders
bbc - news
daily politics: bbc parliament
reporting scotland at 6.30pm
sky nrws
election debate reaction
house of lords
6 oclocknews
it v
benifit programs
game shows
meridian area local itv news
newsmivht
bt sport
bbc2 daily politics show
newsnight (it's crap btw)
head to head
political program
bbc question tme
news & follow ups to the debate
stv news at ten
pawb ai farn
conservative party broadcast
bb2
prime minister questions time
daily tv 

In [ ]:
# [fuzz.WRatio, fuzz.QRatio,
#                   fuzz.token_set_ratio, fuzz.token_sort_ratio,
#                   fuzz.partial_token_set_ratio, fuzz.partial_token_sort_ratio,
#                   fuzz.UWRatio, fuzz.UQRatio]

In [ ]:
# make a choices dict
# include both *crap patterns* and *pertinent patterns*

In [467]:
# start with top 300


In [468]:
df_fuzz = pd.DataFrame(fuzz_dict)
df_fuzz.index = [examples[x] for x in range(288,len(examples))]

In [469]:
df_fuzz.to_csv('../BES_analysis_data/W19_comb_strings\\tv_guesses_pastbase_1.csv')

In [ ]:
## manual processing!!

In [471]:
df_fixed = pd.read_csv('../BES_analysis_data/W19_comb_strings\\tv_guesses_pastbase_1_fixed.csv')

In [472]:
df_fixed = df_fixed.rename(columns={"Unnamed: 0":'original',"Unnamed: 1":'fixed',})

In [473]:
df_fixed[["original","fixed"]][df_fixed["fixed"].notnull()].set_index("original")['fixed'].to_dict()

{'nrws': 'news',
 'day break': 'daybreak',
 'daliy politics': 'daily politics',
 'cameron v milliband': 'cameron vs miliband',
 'kaiser report': 'keiser report',
 'local bbc': 'bbc local news',
 'first leaders debate': 'first debate',
 'leaders debate scotland': 'scottish leaders debate',
 'new at ten': 'news at ten',
 'breakfast on bbc': 'bbc breakfast',
 'news on bbc1': 'bbc1 news',
 'questiom time': 'question time',
 'referendum debates': 'referendum debate',
 'daly politics': 'daily politics',
 'stv leaders debates': 'stv leaders debate',
 'bbc south today': 'south today',
 'itv local': 'itv local news',
 'ten oclock news': 'news at ten',
 'monaghan': 'murnaghan',
 'bbc 6 news': "6 o'clock news",
 'question tme': 'question time',
 'bbc news tv': 'bbc news',
 'part political broadcasts': 'party political broadcasts',
 'bbc scotland leaders debate': 'scottish leaders debate',
 'dayly politics': 'daily politics',
 'bb': 'bbc',
 'andrew neil show': 'andrew neil',
 'bbc nws': 'bbc news'

In [ ]:
### itn -> itv news, itn new -> itv news!

In [ ]:
## dummied versions

In [510]:
tvradiopaperinternet_dummies = pd.DataFrame()

In [495]:

# tvradiopaperinternet[tv_cols] = tvradiopaperinternet[tv_cols].replace({x:np.nan for x in vc[vc<10].index})
# vc[vc>=10].index

Index(['bbc news', 'question time', 'newsnight', 'news', 'sky news',
       'itv news', 'daily politics', 'channel 4 news', 'andrew marr',
       'sunday politics',
       ...
       'gmtv', 'south today', 'trews', 'national debate', 'fox news',
       'scotland 2014', 'referendum debate', 'itv evening news',
       'morning news', 'chase'],
      dtype='object', length=205)

In [501]:
vc = tvradiopaperinternet[tv_cols].stack().value_counts()

temp_df = pd.get_dummies(tvradiopaperinternet[tv_cols].apply(lambda x: x.astype('category').cat.set_categories(list(vc[vc>=10].index))))

for var_no in range(0,len(vc[vc>=10].index)):
    amalg_name = temp_df.columns[var_no].replace("1_1W4","")
    tvradiopaperinternet_dummies[amalg_name] = temp_df[[temp_df.columns[(x*205)+var_no] for x in range(9)]].any(axis=1)

,tv1_1W4_bbc news,tv1_1W4_question time,tv1_1W4_newsnight,tv1_1W4_news,tv1_1W4_sky news,tv1_1W4_itv news,tv1_1W4_daily politics,tv1_1W4_channel 4 news,tv1_1W4_andrew marr,tv1_1W4_sunday politics,...,tv3_1W8_gmtv,tv3_1W8_south today,tv3_1W8_trews,tv3_1W8_national debate,tv3_1W8_fox news,tv3_1W8_scotland 2014,tv3_1W8_referendum debate,tv3_1W8_itv evening news,tv3_1W8_morning news,tv3_1W8_chase
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [514]:
min_sample = 10

vc = tvradiopaperinternet[radio_cols].stack().value_counts()

temp_df = pd.get_dummies(tvradiopaperinternet[radio_cols].apply(lambda x: x.astype('category').cat.set_categories(list(vc[vc>=min_sample].index))))

cat_size = len(vc[vc>=min_sample].index)

for var_no in range(0,cat_size):
    amalg_name = temp_df.columns[var_no].replace("1_1W4","")
    tvradiopaperinternet_dummies[amalg_name] = temp_df[[temp_df.columns[(x*cat_size)+var_no] for x in range(9)]].any(axis=1)

In [516]:
min_sample = 10

vc = tvradiopaperinternet[paper_cols].stack().value_counts()

temp_df = pd.get_dummies(tvradiopaperinternet[paper_cols].apply(lambda x: x.astype('category').cat.set_categories(list(vc[vc>=min_sample].index))))

cat_size = len(vc[vc>=min_sample].index)

for var_no in range(0,cat_size):
    amalg_name = temp_df.columns[var_no].replace("1_1W4","")
    tvradiopaperinternet_dummies[amalg_name] = temp_df[[temp_df.columns[(x*cat_size)+var_no] for x in range(9)]].any(axis=1)

In [517]:
min_sample = 10

vc = tvradiopaperinternet[internet_cols].stack().value_counts()

temp_df = pd.get_dummies(tvradiopaperinternet[internet_cols].apply(lambda x: x.astype('category').cat.set_categories(list(vc[vc>=min_sample].index))))

cat_size = len(vc[vc>=min_sample].index)

for var_no in range(0,cat_size):
    amalg_name = temp_df.columns[var_no].replace("1_1W4","")
    tvradiopaperinternet_dummies[amalg_name] = temp_df[[temp_df.columns[(x*cat_size)+var_no] for x in range(9)]].any(axis=1)

In [520]:
tvradiopaperinternet_dummies.astype('uint8').to_csv('../BES_analysis_data/W19_comb_strings\\tvradiopaperinternet_dummied.csv')


In [132]:
#####
### q1 variable section
# Terrestrial programmes(news/notnews) watched before yesterday/yesterday/today

In [133]:
ser = pd.Series( flatten( df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan).stack().values ) )
ser.value_counts()

#.value_counts().head(60)[-60:]

201911172200|bbc1|bbc news|news                                                    589
201911242200|bbc1|bbc news|news                                                    572
201911192000|itv1|johnson v corbyn: the itv debate|documentary: current affairs    567
201912012200|bbc1|bbc news|news                                                    555
201911221900|bbc1|question time leaders special|current affairs: politics          552
                                                                                  ... 
201912051200|                                                                        1
201911202230|bbc1|regional news and weather                                          1
20191209                                                                             1
201912062030|bbc1|the prime                                                          1
201911152200|bbc1|bbc news at ten|ne                                                 1
Length: 3384, dtype: int64

In [134]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan).stack().value_counts().head(180)[-60:]

201912071720|bbc1|regional news and weather|news                                                                                     52
201911171830|channel 4|channel 4 news|news                                                                                           52
201912090600|bbc1|breakfast|daytime                                                                                                  51
201911231800|itv1|itv evening news|news~201911231815|itv1|regional news and weather|news                                             51
201911242230|itv1|itv news|news                                                                                                      50
201911261800|bbc1|bbc news at six|news~201911261830|bbc1|regional news and weather|news                                              50
201912091300|bbc1|bbc news at one|daytime                                                                                            50
201912011710|bbc1|regional news and weather|news

In [135]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).stack().value_counts()

0    528982
1     25428
2     13861
3      6398
4      3367
5      2155
6      1508
7       336
8         1
dtype: int64

In [136]:
(df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0)==8).sum()

q1_before_yesterdayW18         0
q1_before_yesterday_newsW18    0
q1_yesterdayW18                0
q1_yesterday_newsW18           1
q1_todayW18                    0
q1_today_newsW18               0
dtype: int64

In [137]:
df["q1_yesterday_newsW18"][df["q1_yesterday_newsW18"].replace("__na__",np.nan)\
           .apply(lambda x: len(x.split("~")) if pd.notnull(x) else 0)==8]

id
78922    201912011000|skynews|sky news|news~20191201120...
Name: q1_yesterday_newsW18, dtype: category
Categories (2630, object): [201911130600|bbcnews24|breakfast|news, 201911130600|bbcnews24|breakfast|news~20191113..., 201911130600|bbcnews24|breakfast|news~20191113..., 201911130600|bbcnews24|breakfast|news~20191113..., ..., 20191210900|skynews|all out politics|news~2019..., 20191210900|skynews|all out politics|news~2019..., 20191211700|skynews|kay burley @breakfast|news, __na__]

In [138]:
df.loc[78922,"q1_yesterday_newsW18"].split("~")
# okay - we'll never actually *need* all 8 columns

['201912011000|skynews|sky news|news',
 '201912011200|bbcnews24|bbc news|news',
 '201912011230|bbcnews24|click|news',
 '201912011300|bbcnews24|bbc news|news',
 '201912011100|skynews|sky news|news',
 '201912011400|bbcnews24|bbc news|news',
 '201912011200|skynews|sky news|news',
 '20191']

In [139]:
# ser.value_counts().apply(lambda x: len(x.index.split("|")))
section_no = [len(x.split("|")) for x in ser.value_counts().index]
sec_no = pd.Series(section_no)
sec_no.value_counts()

4    2353
3     780
2     142
1     109
dtype: int64

In [140]:
ser.value_counts().loc[ser.value_counts().index[sec_no>=2]].sort_index().head(60)

201911111800|itv1|regional news and weather|news                                          1
201911112200|bbc1|bbc news at ten|news                                                    1
201911120600|bbc1|breakfast|daytime                                                     175
201911120600|bbcnews24|breakfast|news                                                    14
201911120600|itv1|good morning britain|daytime                                           47
201911120600|skynews|the early rundown|news                                               6
201911120700|skynews|kay burley @breakfast|news                                          34
201911120830|bbcnews24|worklife|news                                                      3
201911120830|itv1|lorraine|daytime                                                        8
201911120900|bbc2|bbc news at nine|daytime                                               13
201911120900|bbcnews24|bbc news at nine|news                                    

In [141]:
ser.apply(lambda x: "|".join(x.split("|")[1:]))[ser.apply(lambda x:len(x.split("|")))>=2].value_counts().sort_index().head(60)

                                                     23
b                                                    13
bb                                                   16
bbc                                                  12
bbc1                                                 12
bbc1|                                                 7
bbc1|ask the leader                                   1
bbc1|ask the leaders|current af                       1
bbc1|ask the leaders|current affairs:                 1
bbc1|ask the leaders|current affairs: politics       59
bbc1|b                                                3
bbc1|bb                                               3
bbc1|bbc                                              3
bbc1|bbc london|news                                 34
bbc1|bbc n                                            5
bbc1|bbc ne                                           5
bbc1|bbc new                                          5
bbc1|bbc news                                   

In [142]:
year_month_day_24hr = np.unique([x for x in ser.apply(lambda x: x.split("|")[0])[ser.apply(lambda x:len(x.split("|")))>=2].value_counts().sort_index().index])

In [143]:
np.unique([x[0:4] for x in year_month_day_24hr])
# all 2019

array(['2019'], dtype='<U4')

In [144]:
np.unique([x[4:6] for x in year_month_day_24hr])
# 11/12

array(['11', '12'], dtype='<U2')

In [145]:
np.unique([x[6:8] for x in year_month_day_24hr])

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30'], dtype='<U2')

In [146]:
np.unique([x[8:12] for x in year_month_day_24hr])

array(['0600', '0700', '0830', '0900', '0915', '0930', '1000', '1030',
       '1100', '1110', '1130', '1200', '1210', '1230', '1300', '1330',
       '1355', '1400', '1430', '1500', '1530', '1600', '1630', '1645',
       '1700', '1705', '1710', '1715', '1720', '1725', '1730', '1735',
       '1745', '1750', '1755', '1800', '1805', '1810', '1815', '1820',
       '1825', '1830', '1835', '1840', '1845', '1850', '1855', '1900',
       '1910', '1915', '1925', '1930', '1945', '1955', '2000', '2005',
       '2030', '2045', '2100', '2105', '2110', '2130', '2200', '2205',
       '2210', '2215', '2220', '2225', '2230', '2235', '2245', '2250',
       '2255', '2300', '2315', '2330', '2345', '600', '700', '830', '900',
       '915', '930'], dtype='<U4')

In [147]:
np.unique([x[0:8] for x in year_month_day_24hr])
# 11/11 - 12/12

array(['20191111', '20191112', '20191113', '20191114', '20191115',
       '20191116', '20191117', '20191118', '20191119', '20191120',
       '20191121', '20191122', '20191123', '20191124', '20191125',
       '20191126', '20191127', '20191128', '20191129', '20191130',
       '20191201', '20191202', '20191203', '20191204', '20191205',
       '20191206', '20191207', '20191208', '20191209', '20191210',
       '20191211', '20191212'], dtype='<U8')

In [148]:
# channel_prog_cat_dict = {"|".join(x.split("|")[0:2]):x.split("|")[2] for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index}
# safe to assume same channel_prog_cat triples?


channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
programmes = ['5 news', '5 news at 5', '5 news lunchtime', '5 news tonight', '5 news weekend', '@ breakfast', 'a week in interviews', 'afternoon live', 'all out politics', 'ask the leaders', 'bbc london', 'bbc news', 'bbc news at 9', 'bbc news at five', 'bbc news at nine', 'bbc news at one', 'bbc news at six', 'bbc news at ten', 'bbc newsroom live', 'beyond 100 days', 'breakfast', 'brexitcast', "britain's care scandal exposed", 'channel 4 news', 'channel 4 news climate debate', 'channel 4 news summary', 'click', 'dateline london', 'election 2019: scotland', 'electioncast', 'fox news @ 9', 'good morning britain', 'ian king live', 'itv evening news', 'itv lunchtime news', 'itv news', 'itv news at ten', 'jeremy vine', 'johnson v corbyn: the itv debate', 'kay burley @breakfast', 'live: britain decides - the everything but brexit debate', 'loose women', 'lorraine', 'lunchtime live', 'macedonia: more than a name', 'mock the week', 'news on the hour', 'newsnight', 'newswatch', 'off air', 'on assignment', 'our world', 'outside source', 'panorama', 'party election broadcast', 'party election broadcast by the brexit party', 'party election broadcast by the conservative party', 'party election broadcast by the green party', 'party election broadcast by the labour party', 'party election broadcast by the liberal democrats', 'party election broadcast on behalf of plaid cymru', 'party election broadcast on behalf of the brexit party', 'party election broadcast on behalf of the conservative party', 'party election broadcast on behalf of the green party', 'party election broadcast on behalf of the labour party', 'party election broadcast on behalf of the liberal democrats', 'party election broadcast on behalf of the scottish national party', 'path to power', 'peston', "pmq's", 'politics live', 'press preview', 'question time', 'question time election special - under 30s', 'question time leaders special', 'regional news and weather', "rivals - america's endgame", 'sky news', 'sky news @ breakfast', 'sky news at 11', 'sky news at 11 with anna botting', 'sky news at six', 'sky news at ten', 'sky news including fyi', 'sky news tonight', 'sky news with sarah hewson', 'sophy ridge on sunday', 'sophy ridge: the take', 'sportsday', 'sportsline', 'talking movies', 'the andrew neil interviews', 'the andrew neil show', 'the bbc election debate', 'the campaign', 'the early rundown', 'the film review', 'the firing line', 'the itv election debate', 'the itv election interviews', 'the leader interviews: boris johnson - tonight', 'the leader interviews: jeremy corbyn - tonight', 'the leader interviews: jo swinson - tonight', 'the leader interviews: nigel farage - tonight', 'the leader interviews: tonight', 'the news hour with mark austin', 'the one show', 'the papers', 'the pledge', 'the prime ministerial debate', 'the prince and the epstein scandal - panorama', 'the real derry girls', 'the richard dimbleby lecture 2019', 'the sarah-jane mee show', 'the travel show', 'through the lens: untold stories', 'turner prize', 'universal credit: one year on - panorama', 'victoria derbyshire', 'victoria derbyshire: your election', 'witness history', 'worklife', 'world news today', 'your questions answered', 'your questions answered: green party', 'your questions answered: nigel farage']
categories = ["comedy","current affairs: politics","daytime","documentary: current affairs","interests","news"]
# 

In [149]:
channel_prog_pairs = ["|".join(x.split("|")[0:2]) for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index]
categories = [x.split("|")[2] for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index]
len(channel_prog_pairs),len(categories),

pd.DataFrame([channel_prog_pairs,categories]).T.groupby(0).sum().to_dict()#[1].values#[1].values

# sadly not a one-for-one channel-prog-prairto category mapping

{1: {'bbc1|ask the leaders': 'current afcurrent affairs:current affairs: politics',
  'bbc1|bbc london': 'news',
  'bbc1|bbc news': 'newnews',
  'bbc1|bbc news at one': 'dadaytime',
  'bbc1|bbc news at six': 'nnenews',
  'bbc1|bbc news at ten': 'nnenewnews',
  'bbc1|breakfast': 'daytime',
  'bbc1|brexitcast': 'ccucurrent affaircurrent affairscurrent affairs:current affairs: politiccurrent affairs: politics',
  'bbc1|panorama': 'documentary',
  'bbc1|party election broadcast by the brexit party': 'currcurrecurrent afcurrent affairs: politicurrent affairs: politics',
  'bbc1|party election broadcast by the conservative party': 'currcurrent acurrent afcurrent affairs:current affairs: polcurrent affairs: policurrent affairs: politcurrent affairs: politics',
  'bbc1|party election broadcast by the green party': 'current affacurrent affaircurrent affairs: pcurrent affairs: politicurrent affairs: politics',
  'bbc1|party election broadcast by the labour party': 'currencurrent afcurrent affair

In [150]:
# np.unique(list(channel_prog_cat_dict.values()))

In [151]:
q1_tv_channels = pd.Series([x.split("|")[1] for x in ser.value_counts().loc[ser.value_counts().index[sec_no>=2]].index])
q1_tv_channels.value_counts()

bbcnews24    788
bbc1         760
itv1         629
skynews      567
bbc2         169
channel 4    129
channel 5    119
              22
it            16
bb            15
b             13
bbc           12
i              6
bbcne          5
channel        3
skyn           3
sk             3
bbcn           2
ch             2
s              2
chan           2
bbcnews        1
bbcnew         1
chann          1
channe         1
c              1
sky            1
itv            1
skynew         1
dtype: int64

In [152]:
for x in q1_tv_channels.value_counts().index:
    if x in channels:
        print(x)
    elif sum([x == y[0:len(x)] for y in channels])==1:
        x = channels[[y[0:len(x)] for y in channels].index(x)]
        print(x)
    else:
        x = np.nan
        print(x)
        

bbcnews24
bbc1
itv1
skynews
bbc2
channel 4
channel 5
nan
itv1
nan
nan
nan
itv1
bbcnews24
nan
skynews
skynews
bbcnews24
nan
skynews
nan
bbcnews24
bbcnews24
nan
nan
nan
skynews
itv1
skynews


In [153]:
ser = pd.Series( flatten( df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan).stack().values ) )
ser.value_counts()



201911172200|bbc1|bbc news|news                                                    589
201911242200|bbc1|bbc news|news                                                    572
201911192000|itv1|johnson v corbyn: the itv debate|documentary: current affairs    567
201912012200|bbc1|bbc news|news                                                    555
201911221900|bbc1|question time leaders special|current affairs: politics          552
                                                                                  ... 
201912051200|                                                                        1
201911202230|bbc1|regional news and weather                                          1
20191209                                                                             1
201912062030|bbc1|the prime                                                          1
201911152200|bbc1|bbc news at ten|ne                                                 1
Length: 3384, dtype: int64

In [154]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).max()

q1_before_yesterdayW18         7
q1_before_yesterday_newsW18    7
q1_yesterdayW18                7
q1_yesterday_newsW18           8
q1_todayW18                    7
q1_today_newsW18               7
dtype: int64

In [155]:
### start building q1 dataframe

In [156]:
# q1_base_cols

In [157]:
q1_col_maxes = df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).max()
q1_base_cols = flatten([[x+"_"+str(y+1) for y in range(q1_col_maxes.loc[x])] for x in q1_col_maxes.index])

In [158]:
q1 = pd.DataFrame(columns = q1_base_cols)

temp = df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan)
for col in q1_col_maxes.index:
    for no in range(1,q1_col_maxes.loc[col]+1):
        q1[col+"_"+str(no)] = temp[col].apply(lambda x: x[no-1] if isinstance(x,list) and len(x)>=no else np.nan)

In [159]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[0] if pd.notnull(x) and len(x.split("|"))>=1 else np.nan)
temp.columns = [x+"_dt" for x in q1_base_cols]
q1[temp.columns]=temp

In [160]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[1] if pd.notnull(x) and len(x.split("|"))>=2 else np.nan)
temp.columns = [x+"_chan" for x in q1_base_cols]
q1[temp.columns]=temp

In [161]:
temp = q1[q1_base_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))>=3 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_cols]
q1[temp.columns]=temp

In [162]:
q1[[x+"_prog" for x in q1_base_cols]].stack().value_counts()

bbc1|regional news and weather                     14155
bbc1|bbc news                                      10114
bbc1|breakfast                                      8471
bbc1|bbc news at ten                                6488
bbc1|bbc news at six                                4958
                                                   ...  
bbcnews24|bbc news at te                               1
itv1|party election broadcast by the green part        1
skynews|pres                                           1
skynews|press previ                                    1
bbc2|p                                                 1
Length: 467, dtype: int64

In [163]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[3] if pd.notnull(x) and len(x.split("|"))>=4 else np.nan)
temp.columns = [x+"_cat" for x in q1_base_cols]
q1[temp.columns]=temp

In [164]:
q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

bbc1         55981
itv1         18786
skynews      11781
bbcnews24    11502
bbc2          4562
channel 4     4147
channel 5      712
                23
it              18
bb              16
b               13
bbc             12
i                6
bbcne            5
channel          4
skyn             4
sk               3
ch               2
chan             2
s                2
bbcn             2
sky              1
chann            1
bbcnews          1
skynew           1
channe           1
c                1
itv              1
bbcnew           1
dtype: int64

In [165]:
# vc = q1[[x+"_dt" for x in q1_base_cols]].applymap(lambda x:fix_q1_date_string(x)).stack().value_counts()

In [166]:
# vc.loc[[x for x in vc.index if len(x)<=11]]

In [167]:
def fix_q1_date_string(x):
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif len(x)==12:
        # it's definitely fine
        return(x)
    elif len(x)==11:
        # it's maybe fine if it's a short 2400 thing - e.g. 600, 915
        # but if the 24 hour string starts with 0/1/2 more reasonable to assume evening than morning
        if x[8] in ["0","1","2"]:
            x=x+"0"
        return(x)
    elif len(x)==10:
        # technically, this could be short or long 24 hour strings
        # but I eyeballed all instances and saw it was only 'long' 24 hour strings
        # some of these should also actually be 15/45 mins past the hour ...
        # but effectively rounding down to the nearest hour seems like an acceptable fix
        return(x+"00")
    else:
        # bugs me to throw away partial information
        # but you'd need to have a good reason to have to add in the extra work
        # needed to handle partially complete datestrings
        return(np.nan)

In [168]:
q1[[x+"_dt" for x in q1_base_cols]] = q1[[x+"_dt" for x in q1_base_cols]].applymap(lambda x:fix_q1_date_string(x))

q1[[x+"_dt" for x in q1_base_cols]].stack().value_counts()

201911181800    782
201911141800    721
201912021800    720
201911291800    714
201911221900    713
               ... 
201912121700      1
201912081230      1
201912112000      1
201911142340      1
201912052240      1
Length: 992, dtype: int64

In [169]:

# q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

In [170]:
def fix_q1_channel_string(x):
    channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in channels:
        # it's definitely fine
        return(x)
    elif sum([x == y[0:len(x)] for y in channels])==1:
        # unique match on what there is to one of the channel strings
        return channels[[y[0:len(x)] for y in channels].index(x)]
    else:
        # in theory, we could match to broader classes (e.g. "bbc" and "channel 4/5")
        # breaks my heart to throw data away!
        # but that would inconsistent with treamtent of the date strings and create extra baggage of overlapping categories
        # waaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa!
        # maybe 40-50 instances
        return np.nan

In [171]:
q1[[x+"_chan" for x in q1_base_cols]] = q1[[x+"_chan" for x in q1_base_cols]].applymap(lambda x:fix_q1_channel_string(x))
q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

bbc1         55981
itv1         18811
skynews      11792
bbcnews24    11511
bbc2          4562
channel 4     4147
channel 5      712
dtype: int64

In [172]:
# q1[[x+"_cat" for x in q1_base_cols]].stack().value_counts()

In [173]:
def fix_q1_news_cat_string(x):
#     channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    else:
        # anything other than empty cell should be "news"
        # at least, going on the distribution I see in the data
        # arguably, this makes this column unnecessary
        return "news"


In [174]:
q1_base_news_cols = [x for x in q1_base_cols if "news" in x]

q1[[x+"_cat" for x in q1_base_news_cols]] = q1[[x+"_cat" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_cat_string(x))

q1[[x+"_cat" for x in q1_base_news_cols]].stack().value_counts()

news    23152
dtype: int64

In [175]:
q1_categories = ['news', 'daytime', 'current affairs','current affairs: politics', 'interests',
        'documentary','documentary: current affairs', 'comedy', ]
q1_base_notnews_cols = [x for x in q1_base_cols if "news" not in x]
q1[[x+"_cat" for x in q1_base_notnews_cols]].stack().value_counts()


news                            49975
daytime                         24177
current affairs: politics        4943
interests                        1336
current affairs                  1223
documentary: current affairs      742
documentary                       368
comedy                             66
new                                57
                                   36
ne                                 25
n                                  24
current af                         24
intere                             23
current a                          20
current affairs:                   19
curr                               19
curre                              15
cu                                 14
current affa                       14
current affairs: p                 13
current affai                      11
current affairs: politi            10
current affairs: poli               9
current affair                      9
current                             9
current affa

In [176]:
def fix_q1_notnews_cat_string(x):
    q1_categories = ['news', 'daytime', 'current affairs','current affairs: politics', 'interests',
        'documentary','documentary: current affairs', 'comedy', ]
    # sorted by generality/frequency
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_categories:
        # it's definitely fine
        return(x)
    elif sum([x == y[0:len(x)] for y in q1_categories])>=1:
        # since this data is somewhat subjective, I'm allowing 'greedy matches'
        # e.g. 'docu' -> 'documentary', 'c' -> 'current affairs'
        return q1_categories[[y[0:len(x)] for y in q1_categories].index(x)]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [177]:
q1[[x+"_cat" for x in q1_base_notnews_cols]] = q1[[x+"_cat" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_cat_string(x))
q1[[x+"_cat" for x in q1_base_notnews_cols]].stack().value_counts()

news                            50117
daytime                         24206
current affairs: politics        5015
current affairs                  1381
interests                        1373
documentary: current affairs      751
documentary                       382
comedy                             66
dtype: int64

In [178]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()

bbc1|regional news and weather         14155
bbc1|bbc news                          10114
bbc1|breakfast                          8471
bbc1|bbc news at ten                    6488
bbc1|bbc news at six                    4958
                                       ...  
bbc1|universal cred                        1
itv1|the leader interviews: nigel f        1
channel 5|party election broadca           1
itv1|regiona                               1
bbc2|p                                     1
Length: 337, dtype: int64

In [179]:
temp = q1[q1_base_notnews_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))==4 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_notnews_cols]
channel_prog_pairs_notnews_vc = temp.stack().value_counts()
channel_prog_pairs_notnews_vc
# 79

bbc1|regional news and weather                                                 14144
bbc1|bbc news                                                                  10108
bbc1|breakfast                                                                  8471
bbc1|bbc news at ten                                                            6484
bbc1|bbc news at six                                                            4956
                                                                               ...  
channel 5|party election broadcast on behalf of the green party                    9
bbc2|your questions answered: nigel farage                                         8
bbc2|your questions answered: green party                                          8
channel 5|party election broadcast on behalf of the scottish national party        7
channel 5|party election broadcast on behalf of the brexit party                   5
Length: 79, dtype: int64

In [180]:
temp = q1[q1_base_news_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))==4 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_news_cols]
channel_prog_pairs_news_vc = temp.stack().value_counts()
channel_prog_pairs_news_vc
# 63

bbcnews24|bbc news                            3034
skynews|press preview                         1858
skynews|all out politics                      1335
bbcnews24|the papers                          1301
skynews|sky news                              1245
                                              ... 
bbcnews24|off air                                2
bbcnews24|rivals - america's endgame             2
bbcnews24|through the lens: untold stories       2
bbcnews24|turner prize                           1
bbcnews24|the firing line                        1
Length: 63, dtype: int64

In [181]:
def fix_q1_notnews_chan_prog_string(x):
    q1_chan_prog_pairs = ['bbc1|regional news and weather', 'bbc1|bbc news', 'bbc1|breakfast', 'bbc1|bbc news at ten', 'bbc1|bbc news at six', 'itv1|regional news and weather', 'bbc1|bbc news at one', 'itv1|itv evening news', 'channel 4|channel 4 news', 'itv1|good morning britain', 'itv1|itv news', 'bbc2|newsnight', 'bbc1|the one show', 'bbc2|politics live', 'bbc1|the andrew neil interviews', 'itv1|itv lunchtime news', 'itv1|itv news at ten', 'bbc1|question time', 'bbc1|question time leaders special', 'itv1|johnson v corbyn: the itv debate', 'itv1|loose women', 'itv1|lorraine', 'bbc1|the prime ministerial debate', 'bbc2|bbc newsroom live', 'bbc2|victoria derbyshire', 'channel 4|party election broadcast', 'bbc1|the bbc election debate', 'channel 4|channel 4 news summary', 'channel 5|jeremy vine', 'itv1|the itv election debate', 'bbc1|party election broadcast by the labour party', 'bbc1|party election broadcast by the conservative party', 'bbc1|question time election special - under 30s', 'itv1|party election broadcast by the liberal democrats', 'itv1|party election broadcast by the labour party', 'bbc1|party election broadcast by the liberal democrats', 'itv1|party election broadcast by the brexit party', 'itv1|party election broadcast by the conservative party', 'bbc1|brexitcast', 'bbc2|bbc news at nine', 'bbc1|the prince and the epstein scandal - panorama', 'channel 5|5 news at 5', 'itv1|the leader interviews: jeremy corbyn - tonight', 'itv1|party election broadcast by the green party', 'itv1|the itv election interviews', 'itv1|the leader interviews: boris johnson - tonight', 'itv1|peston', 'bbc2|the andrew neil show', 'bbc2|bbc news at 9', 'itv1|the leader interviews: nigel farage - tonight', 'channel 4|channel 4 news climate debate', 'channel 5|5 news tonight', 'itv1|the leader interviews: tonight', 'bbc1|party election broadcast by the green party', 'itv1|the leader interviews: jo swinson - tonight', 'bbc1|party election broadcast by the brexit party', 'channel 5|5 news weekend', 'bbc1|panorama', 'bbc2|mock the week', 'bbc1|ask the leaders', 'channel 4|live: britain decides - the everything but brexit debate', 'bbc2|the one show', 'bbc1|the richard dimbleby lecture 2019', 'channel 5|5 news lunchtime', 'bbc1|universal credit: one year on - panorama', 'bbc1|bbc london', 'channel 5|5 news', 'bbc2|victoria derbyshire: your election', 'channel 5|party election broadcast on behalf of plaid cymru', 'channel 5|party election broadcast on behalf of the labour party', 'bbc2|your questions answered', 'channel 5|party election broadcast on behalf of the conservative party', 'itv1|on assignment', 'channel 5|party election broadcast on behalf of the liberal democrats', 'channel 5|party election broadcast on behalf of the green party', 'bbc2|your questions answered: nigel farage', 'bbc2|your questions answered: green party', 'channel 5|party election broadcast on behalf of the scottish national party', 'channel 5|party election broadcast on behalf of the brexit party']
    # sorted by frequency
    q1_chan_prog_generic_dict = {"itv1|the leader interviews: ":"the leader interviews(amb)",
                                 "bbc1|party election broadcast by the ":"party election broadcast(amb)",
                                 "itv1|party election broadcast by the ":"party election broadcast(amb)",
                                 "channel 5|party election broadcast on behalf of the ":"party election broadcast(amb)",
                                 "itv1|itv news":"itv news(amb)",
                                 'bbc1|bbc news at ':"bbc news(amb)",                                 
                                 'channel 4|channel 4 news':"channel 4 news(amb)",
                                 'channel 5|5 news':"channel 5 news(amb)",
                                 'bbc1|question time':'question time(amb)'}
    
    
    q1_chan_prog_generic_keys = list(q1_chan_prog_generic_dict.keys())
    # because I wept at the thought of throwing away so many ambiguous cases
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_chan_prog_pairs:
        # it's definitely fine
        return x.split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_pairs])==1:
        # I think we need to stick to unambiguous matches here
        # it is *theoretically* possible that we'd be able to disambiguate some of these by including the dates too!
        return q1_chan_prog_pairs[[y[0:len(x)] for y in q1_chan_prog_pairs].index(x)].split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_generic_keys])==1:
        # because 200+ ambiguous cases seemed too much -> got it down to 42
        return q1_chan_prog_generic_dict[q1_chan_prog_generic_keys[[y[0:len(x)] for y in q1_chan_prog_generic_keys].index(x)]]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [182]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x)).stack().value_counts()

regional news and weather                                            19357
bbc news                                                             10114
breakfast                                                             8471
bbc news at ten                                                       6504
bbc news at six                                                       4960
                                                                     ...  
your questions answered: nigel farage                                    8
party election broadcast on behalf of the scottish national party        7
party election broadcast on behalf of the brexit party                   5
question time(amb)                                                       4
channel 5 news(amb)                                                      2
Length: 79, dtype: int64

In [183]:
temp = q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x))
entries_dropped = (q1[[x+"_prog" for x in q1_base_notnews_cols]].stack(dropna=False).notnull() & temp.stack(dropna=False).isnull())
entries_dropped.sum()

28

In [184]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack(dropna=False)[entries_dropped].value_counts().head(60)

itv1|                    11
bbc1|                     7
bbc1|th                   2
bbc1|the                  2
itv1|the itv              1
bbc2|                     1
bbc2|bbc new              1
itv1|the i                1
bbc1|the p                1
itv1|the itv election     1
dtype: int64

In [185]:
q1[[x+"_prog" for x in q1_base_notnews_cols]] = q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x))
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()

regional news and weather                                            19357
bbc news                                                             10114
breakfast                                                             8471
bbc news at ten                                                       6504
bbc news at six                                                       4960
                                                                     ...  
your questions answered: nigel farage                                    8
party election broadcast on behalf of the scottish national party        7
party election broadcast on behalf of the brexit party                   5
question time(amb)                                                       4
channel 5 news(amb)                                                      2
Length: 79, dtype: int64

In [186]:
# # woops!
# q1[[x+"_prog" for x in q1_base_notnews_cols]] = q1[[x+"_prog" for x in q1_base_notnews_cols]].replace("bbc1|question time(amb)","question time(amb)")
# q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()


In [187]:
channel_prog_pairs_news_vc

bbcnews24|bbc news                            3034
skynews|press preview                         1858
skynews|all out politics                      1335
bbcnews24|the papers                          1301
skynews|sky news                              1245
                                              ... 
bbcnews24|off air                                2
bbcnews24|rivals - america's endgame             2
bbcnews24|through the lens: untold stories       2
bbcnews24|turner prize                           1
bbcnews24|the firing line                        1
Length: 63, dtype: int64

In [188]:
# list(channel_prog_pairs_news_vc.index)

In [189]:
def fix_q1_news_chan_prog_string(x):
    q1_chan_prog_pairs = ['bbcnews24|bbc news', 'skynews|press preview', 'skynews|all out politics', 'bbcnews24|the papers', 'skynews|sky news', 'skynews|sky news at ten', 'bbcnews24|breakfast', 'skynews|kay burley @breakfast', 'bbcnews24|bbc news at six', 'skynews|sky news tonight', 'bbcnews24|afternoon live', 'bbcnews24|bbc news at one', 'bbcnews24|bbc newsroom live', 'bbcnews24|bbc news at ten', 'skynews|sophy ridge on sunday', 'skynews|the sarah-jane mee show', 'bbcnews24|victoria derbyshire', 'bbcnews24|bbc news at nine', 'skynews|lunchtime live', 'bbcnews24|beyond 100 days', 'skynews|sky news @ breakfast', 'skynews|the early rundown', 'bbcnews24|bbc news at five', 'skynews|ian king live', 'skynews|sky news at six', 'skynews|the campaign', 'bbcnews24|sportsday', 'bbcnews24|electioncast', 'skynews|sky news at 11', 'skynews|path to power', 'skynews|@ breakfast', 'skynews|the news hour with mark austin', 'bbcnews24|outside source', 'bbcnews24|world news today', 'bbcnews24|worklife', 'skynews|sophy ridge: the take', 'skynews|sky news with sarah hewson', 'bbcnews24|dateline london', 'bbcnews24|click', 'skynews|sky news including fyi', 'skynews|a week in interviews', 'skynews|sky news at 11 with anna botting', 'bbcnews24|the film review', 'bbcnews24|brexitcast', 'bbcnews24|the travel show', 'skynews|news on the hour', 'skynews|sportsline', 'skynews|the pledge', 'bbcnews24|newswatch', 'skynews|fox news @ 9', 'bbcnews24|our world', "skynews|pmq's", 'bbcnews24|talking movies', 'bbcnews24|witness history', 'bbcnews24|election 2019: scotland', "bbcnews24|britain's care scandal exposed", 'bbcnews24|the real derry girls', 'bbcnews24|macedonia: more than a name', "bbcnews24|rivals - america's endgame", 'bbcnews24|through the lens: untold stories', 'bbcnews24|off air', 'bbcnews24|the firing line', 'bbcnews24|turner prize']
    # sorted by frequency
    q1_chan_prog_generic_dict = {"bbcnews24|bbc news at ":"bbc news(amb)",
                                 "skynews|sky news at ":"sky news(amb)",
                                }
    
    q1_chan_prog_generic_keys = list(q1_chan_prog_generic_dict.keys())
    # because I wept at the thought of throwing away so many ambiguous cases
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_chan_prog_pairs:
        # it's definitely fine
        return x.split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_pairs])==1:
        # I think we need to stick to unambiguous matches here
        # it is *theoretically* possible that we'd be able to disambiguate some of these by including the dates too!
        return q1_chan_prog_pairs[[y[0:len(x)] for y in q1_chan_prog_pairs].index(x)].split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_generic_keys])==1:
        # because 50+ ambiguous cases seemed too much -> got it down to 4
        return q1_chan_prog_generic_dict[q1_chan_prog_generic_keys[[y[0:len(x)] for y in q1_chan_prog_generic_keys].index(x)]]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [190]:
q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x)).stack().value_counts()

bbc news                            3037
press preview                       1871
all out politics                    1336
the papers                          1306
sky news                            1247
                                    ... 
rivals - america's endgame             2
off air                                2
through the lens: untold stories       2
the firing line                        1
turner prize                           1
Length: 65, dtype: int64

In [191]:
temp = q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x))
entries_dropped = (q1[[x+"_prog" for x in q1_base_news_cols]].stack(dropna=False).notnull() & temp.stack(dropna=False).isnull())
entries_dropped.sum()

5

In [192]:
q1[[x+"_prog" for x in q1_base_news_cols]].stack(dropna=False)[entries_dropped].value_counts().head(60)

bbcnews24|th         2
bbcnews24|the        1
skynews|sophy rid    1
skynews|the          1
dtype: int64

In [193]:
q1[[x+"_prog" for x in q1_base_news_cols]] = q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x))
q1[[x+"_prog" for x in q1_base_news_cols]].stack().value_counts()

bbc news                            3037
press preview                       1871
all out politics                    1336
the papers                          1306
sky news                            1247
                                    ... 
rivals - america's endgame             2
off air                                2
through the lens: untold stories       2
the firing line                        1
turner prize                           1
Length: 65, dtype: int64

In [194]:
### q1 stuff ... almost done!

def turn_datestrings_to_dates(x):
    if pd.isnull(x):
        return pd.NaT
    elif len(x)==11:
        dstr = list(x)
        dstr.insert(8,"0")
        dstr = "".join(dstr)
        return pd.to_datetime(dstr,format='%Y%m%d%H%M')
    else:
        return pd.to_datetime(x,format='%Y%m%d%H%M')

        

In [195]:
# slowish ~ 1 min - could speed up by prepping columns and processing all at once
temp = q1[[x+"_dt" for x in q1_base_cols]].applymap( lambda x: turn_datestrings_to_dates(x) )

In [196]:
temp["q1_today_newsW18_7_dt"].value_counts().sort_index()

2019-11-17 23:00:00    1
2019-11-22 22:40:00    1
2019-11-23 22:30:00    1
2019-11-29 23:30:00    1
2019-11-30 17:00:00    1
2019-12-02 21:00:00    1
2019-12-08 13:00:00    1
2019-12-11 18:00:00    1
Name: q1_today_newsW18_7_dt, dtype: int64

In [197]:
q1[[x+"_dt" for x in q1_base_cols]] = q1[[x+"_dt" for x in q1_base_cols]].applymap( lambda x: turn_datestrings_to_dates(x) )

In [198]:
q1.dropna(how='all')

,q1_before_yesterdayW18_1,q1_before_yesterdayW18_2,q1_before_yesterdayW18_3,q1_before_yesterdayW18_4,q1_before_yesterdayW18_5,q1_before_yesterdayW18_6,q1_before_yesterdayW18_7,q1_before_yesterday_newsW18_1,q1_before_yesterday_newsW18_2,q1_before_yesterday_newsW18_3,...,q1_todayW18_5_cat,q1_todayW18_6_cat,q1_todayW18_7_cat,q1_today_newsW18_1_cat,q1_today_newsW18_2_cat,q1_today_newsW18_3_cat,q1_today_newsW18_4_cat,q1_today_newsW18_5_cat,q1_today_newsW18_6_cat,q1_today_newsW18_7_cat
id,,,,,,,,,,,,,,,,,,,,,
9,201912041800|bbc1|bbc news|news,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,201911291900|bbc1|the bbc election debate|news,201911292200|bbc1|bbc news at ten|news,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,20191209600|bbc1|breakfast|daytime,201912092200|bbc1|bbc news at ten|news,201912092230|itv1|regional news and weather|news,201912091800|itv1|regional news and weather|news,NaN,NaN,NaN,201912091900|skynews|sky news tonight|news,20191209600|bbcnews24|breakfast|news,201912092000|bbcnews24|bbc news|news,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201912011000|skynews|sky news|news,NaN,NaN,...,NaN,NaN,NaN,news,NaN,NaN,NaN,NaN,NaN,NaN
23,201911221900|bbc1|question time leaders specia...,201911222200|bbc1|bbc news at ten|news,201911221300|bbc1|bbc news at one|daytime,201911221330|bbc1|regional news and weather|da...,NaN,NaN,NaN,201911220600|bbcnews24|breakfast|news,NaN,NaN,...,NaN,NaN,NaN,news,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98327,201912012200|bbc1|bbc news|news,201912011900|itv1|the itv election debate|curr...,NaN,NaN,NaN,NaN,NaN,201912010600|skynews|sky news @ breakfast|news,201912010830|skynews|sophy ridge on sunday|news,201912010930|bbcnews24|the papers|news,...,NaN,NaN,NaN,news,news,NaN,NaN,NaN,NaN,NaN
98332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201911160600|bbcnews24|breakfast|news,201911161300|bbcnews24|bbc news|news,201911162200|bbcnews24|bbc news|news,...,NaN,NaN,NaN,news,news,news,news,NaN,NaN,NaN
98333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201911122200|bbcnews24|bbc news at ten|news,201911121800|bbcnews24|bbc news at six|news,201911122245|bbcnews24|the papers|news,...,NaN,NaN,NaN,news,news,news,NaN,NaN,NaN,NaN


In [199]:
q1.dtypes.value_counts()

object            166
datetime64[ns]     43
float64             6
dtype: int64

In [200]:
# drop the completely blank 8th quadruple
q1.drop(['q1_yesterday_newsW18_8_dt', 'q1_yesterday_newsW18_8_chan',
       'q1_yesterday_newsW18_8_prog',  'q1_yesterday_newsW18_8_cat',  ],axis=1,inplace=True)

In [201]:
# what to do about these?
# can probably infer these *reasonably* based on most common channel:prog:cat triples


#'q1_before_yesterdayW18_7_cat', 'q1_yesterdayW18_7_cat', 'q1_todayW18_7_cat'

In [202]:
list( q1.columns[ q1.dtypes == 'float64' ] )

['q1_before_yesterdayW18_7_cat', 'q1_yesterdayW18_7_cat', 'q1_todayW18_7_cat']

In [203]:
q1.dtypes.value_counts()

object            166
datetime64[ns]     42
float64             3
dtype: int64

In [204]:
q1[q1.columns[ q1.dtypes!="datetime64[ns]" ]] = q1[q1.columns[ q1.dtypes!="datetime64[ns]" ]].replace(" ",np.nan).astype('category')

In [205]:
q1.dtypes.value_counts()

datetime64[ns]    42
category          21
category          21
category          18
category          13
                  ..
category           1
category           1
category           1
category           1
category           1
Length: 96, dtype: int64

In [206]:
# replace(" ",np.nan)
# shared categories explicitly defined

In [207]:
# q1.to_pickle('../BES_analysis_data/W19_comb_strings\\q1.zip', compression='zip')

In [208]:
q1[search(q1,"_prog").index].dropna(how='all').stack().value_counts()

regional news and weather           19357
bbc news                            13151
breakfast                            9490
bbc news at ten                      7045
bbc news at six                      5605
                                    ...  
through the lens: untold stories        2
rivals - america's endgame              2
off air                                 2
turner prize                            1
the firing line                         1
Length: 134, dtype: int64

In [209]:
q1[search(q1,"before_yesterday.*_dt").index]#.dropna(how='any').apply(lambda x: x.dt.day).var(axis=1)#.stack().value_counts()

,q1_before_yesterdayW18_1_dt,q1_before_yesterdayW18_2_dt,q1_before_yesterdayW18_3_dt,q1_before_yesterdayW18_4_dt,q1_before_yesterdayW18_5_dt,q1_before_yesterdayW18_6_dt,q1_before_yesterdayW18_7_dt,q1_before_yesterday_newsW18_1_dt,q1_before_yesterday_newsW18_2_dt,q1_before_yesterday_newsW18_3_dt,q1_before_yesterday_newsW18_4_dt,q1_before_yesterday_newsW18_5_dt,q1_before_yesterday_newsW18_6_dt,q1_before_yesterday_newsW18_7_dt
id,,,,,,,,,,,,,,
1,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
98448,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
98449,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [210]:
# note - before_yesterday = seems to equal 1 specific day, ditto yesterday, today

In [211]:
# One last thing - the non-string combined dataset had a set of variables which
# really really *look* like they are the corresponding date variables for


# xbefore_yesterday_dateW18    59181.0
# xtoday_dateW18               59181.0
# xyesterday_dateW18           59181.0

# these three variables are always consecutive days (i.e. wasteful for storage!)

# but do they match up with the q1 variable data?

In [212]:
BES_Panel["xbefore_yesterday_dateW18"].notnull().sum()

37825

In [213]:
BES_Panel[["xbefore_yesterday_dateW18","xyesterday_dateW18","xtoday_dateW18"]].dropna(how='any').diff(axis=1)["xtoday_dateW18"].value_counts()

1 days    37825
Name: xtoday_dateW18, dtype: int64

In [214]:
test_df = pd.DataFrame()
test_df["xbefore_yesterday_dateW18"] = BES_Panel["xbefore_yesterday_dateW18"]
test_df["q1_before_yesterdayW18_1_dt"] = q1["q1_before_yesterdayW18_1_dt"] 

# not missing values in 'xbefore_yesterday_dateW18' not also missing value in 'q1_before_yesterdayW18_1_dt'
# but plenty of missing values other way around (also, size of non-null looks about right for full sample of W18)

test_df.dropna(how='any').diff(axis=1).apply(lambda x: x.dt.days)["q1_before_yesterdayW18_1_dt"].value_counts()
# bingo - same day each time

0    16521
Name: q1_before_yesterdayW18_1_dt, dtype: int64

In [215]:
q1[search(q1,"before_yesterday.*_dt").index].apply(lambda x: x-BES_Panel["xbefore_yesterday_dateW18"]).dropna(how='all').stack().apply(lambda x: x.days).value_counts()

0    47483
dtype: int64

In [216]:
q1[search(q1,"q1_yesterday.*_dt").index].apply(lambda x: x-BES_Panel["xyesterday_dateW18"]).dropna(how='all').stack().apply(lambda x: x.days).value_counts()

0    38906
dtype: int64

In [217]:
q1[search(q1,"q1_today.*_dt").index].apply(lambda x: x-BES_Panel["xtoday_dateW18"]).dropna(how='all').stack().apply(lambda x: x.days).value_counts()

0    21263
dtype: int64

In [218]:
## okay - all days line up perfectly

# we actually only need one *date* signifier (or, three, for the convenience of not adding)
# date variables can be reduced to 'hour' variables

# NOTE - entries are all within same day, but **no guaranteed chronological order** to entries

In [219]:
q1[search(q1,"q1_today.*_dt").index].apply(lambda x: x-BES_Panel["xtoday_dateW18"]).dropna(how='all').stack().value_counts()

# 75 ... not so large a set

hrs_list = q1[search(q1,"q1_today.*_dt").index].apply(lambda x: x-BES_Panel["xtoday_dateW18"]).dropna(how='all').stack()
hrs_list.append( q1[search(q1,"q1_yesterday.*_dt").index].apply(lambda x: x-BES_Panel["xyesterday_dateW18"]).dropna(how='all').stack() )
hrs_list.append(q1[search(q1,"before_yesterday.*_dt").index].apply(lambda x: x-BES_Panel["xbefore_yesterday_dateW18"]).dropna(how='all').stack() )

id                                     
11     q1_todayW18_1_dt                   22:20:00
       q1_todayW18_2_dt                   22:00:00
       q1_todayW18_3_dt                   19:00:00
14     q1_today_newsW18_1_dt              09:00:00
23     q1_todayW18_1_dt                   17:05:00
                                            ...   
98333  q1_before_yesterday_newsW18_3_dt   22:45:00
98339  q1_before_yesterdayW18_1_dt        22:00:00
98343  q1_before_yesterdayW18_1_dt        18:00:00
       q1_before_yesterdayW18_2_dt        22:00:00
       q1_before_yesterdayW18_3_dt        18:30:00
Length: 68746, dtype: timedelta64[ns]

In [220]:
hrs_list.value_counts()

# still only 75 different hour times (does make it look like there's a pulldown list)

06:00:00    4065
18:00:00    2539
18:30:00    1832
22:00:00    1707
13:00:00    1396
            ... 
17:50:00       2
20:05:00       1
22:40:00       1
19:10:00       1
21:05:00       1
Length: 75, dtype: int64

In [221]:
# replace _dt with _hr? and add three date variables

In [222]:
q1[ [x.replace("_dt","_hr") for x in search(q1,"q1_today.*_dt").index] ] = q1[search(q1,"q1_today.*_dt").index].applymap(lambda x: x.time() if pd.notnull(x) else pd.NaT)

In [223]:
q1[ [x.replace("_dt","_hr") for x in search(q1,"q1_yesterday.*_dt").index] ] = q1[search(q1,"q1_yesterday.*_dt").index].applymap(lambda x: x.time() if pd.notnull(x) else pd.NaT)

In [224]:
q1[ [x.replace("_dt","_hr") for x in search(q1,"q1_before_yesterday.*_dt").index] ] = q1[search(q1,"q1_before_yesterday.*_dt").index].applymap(lambda x: x.time() if pd.notnull(x) else pd.NaT)

In [225]:
# xbefore_yesterday_dateW18    59181.0
# xtoday_dateW18               59181.0
# xyesterday_dateW18           59181.0

q1["xbefore_yesterday_dateW18"] = BES_Panel["xbefore_yesterday_dateW18"]
q1["xtoday_dateW18"]            = BES_Panel["xtoday_dateW18"]
q1["xyesterday_dateW18"]        = BES_Panel["xyesterday_dateW18"]

In [226]:
q1.drop([x for x in q1.columns if "_dt" in x],axis=1,inplace=True)

In [227]:
q1

# so, technically, it's a bit of a waste including anything but wave 18 rows ...

# ... does it matter, spacewise?

# ... eh - still a problem with 477 indices that probably shouldn't be in the dataset, just grab it all

,q1_before_yesterdayW18_1,q1_before_yesterdayW18_2,q1_before_yesterdayW18_3,q1_before_yesterdayW18_4,q1_before_yesterdayW18_5,q1_before_yesterdayW18_6,q1_before_yesterdayW18_7,q1_before_yesterday_newsW18_1,q1_before_yesterday_newsW18_2,q1_before_yesterday_newsW18_3,...,q1_before_yesterday_newsW18_1_hr,q1_before_yesterday_newsW18_2_hr,q1_before_yesterday_newsW18_3_hr,q1_before_yesterday_newsW18_4_hr,q1_before_yesterday_newsW18_5_hr,q1_before_yesterday_newsW18_6_hr,q1_before_yesterday_newsW18_7_hr,xbefore_yesterday_dateW18,xtoday_dateW18,xyesterday_dateW18
id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-17,2019-11-19,2019-11-18
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
98448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
98449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [228]:
q1.to_pickle('../BES_analysis_data/W19_comb_strings\\q1.zip', compression='zip')

In [229]:
q1.to_csv('../BES_analysis_data/W19_comb_strings\\q1.csv')

In [230]:
discrepant_ids = BES_Panel.index[(q1.notnull().any(axis=1) != BES_Panel["wave18"])]

In [231]:
(BES_Panel[match(BES_Panel,"wave").index].sum(axis=1)==0).sum()

477

In [232]:
BES_Panel.loc[discrepant_ids,].reset_index()["id"].to_csv('../BES_analysis_data/W19_comb_strings\\discrepant_ids.csv')

In [233]:
BES_Panel.shape, q1.shape

((97006, 8079), (97006, 214))

In [234]:
BES_Panel[match(BES_Panel,"wave").index].any(axis=1).sum()

96529

In [235]:
BES_Panel.loc[discrepant_ids,match(BES_Panel,"wt_new").index].notnull().sum()

wt_new_W10           0
wt_new_W11           0
wt_new_W14           0
wt_new_W15           0
wt_new_W16           0
wt_new_W1_W16        0
wt_new_W18           0
wt_new_W6W7          0
wt_new_W2W6          0
wt_new_W2W6W7        0
wt_new_W9_W13        0
wt_new_W12           0
wt_new_W1_W11        0
wt_new_W1_W12        0
wt_new_W1_W13        0
wt_new_W6_W11        0
wt_new_W6_W12        0
wt_new_W6_W13        0
wt_new_W11_W13       0
wt_new_W13W16        0
wt_new_W13_result    0
wt_new_W6W16         0
wt_new_W2W16         0
wt_new_W17           0
wt_new_W13W17        0
wt_new_W6W13W19      0
wt_new_W9W19         0
wt_new_W13_W19       0
wt_new_W13W19        0
wt_new_W16W19        0
wt_new_W17W18W19     0
wt_new_W19_result    0
wt_new_W9            0
wt_new_W8            0
wt_new_W7            0
wt_new_W6            0
wt_new_W13W14        0
wt_new_W9_W14        0
wt_new_W6_W14        0
wt_new_W1_W14        0
wt_new_W6W13         0
wt_new_W7W13         0
wt_newW17W19         0
dtype: int6

In [236]:
BES_Panel.loc[discrepant_ids].notnull().sum().sort_values().tail(50)

econPersonalRetroW6       0
preferMPDegreeW6          0
preferMPMuslimW6          0
preferMPLGBTW6            0
preferMPChristW6          0
preferMPEthnicW6          0
preferMPYoungW6           0
preferMPDisabledW6        0
preferMPWomenW6           0
preferMPWorkClassW6       0
preferMPLocalW6           0
partyHelpMethod_5W6       0
partyHelpMethod_4W6       0
partyHelpMethod_3W6       0
partyHelpMethod_2W6       0
twitterUseW6              0
ethno3W7                  0
twitterInfof2fW6          0
fbInfof2fW6               0
ethno1W7                  0
europeannessW7            0
englishnessW7             0
welshnessW7               0
scottishnessW7            0
britishnessW7             0
pidPraiseGoodW7           0
pidMyPartyW7              0
pidConnectedW7            0
pidRuinDayW7              0
pidCommonPartyW7          0
pidCriticisePartyW7       0
pidInterestedOthersW7     0
fbUseW6                   0
pidWeTheyW7               0
euRefFinalW7              0
euRefExpectationW7  

In [237]:
search(BES_Panel,"p_paper")

p_paper_readW1     30383
p_paper_readW2     30239
p_paper_readW3     28122
p_paper_readW4     31442
p_paper_readW5     30829
p_paper_readW6     30149
p_paper_readW7     30841
p_paper_readW8     33500
p_paper_readW9     30034
p_paper_readW10    30236
p_paper_readW11    30955
p_paper_readW12    34393
p_paper_readW13    31135
p_paper_readW14    31061
p_paper_readW15    30840
p_paper_readW16    37959
p_paper_readW17    34365
p_paper_readW18    37824
p_paper_readW19    32176
dtype: int64

regional news and weather           19357
bbc news                            13151
breakfast                            9490
bbc news at ten                      7045
bbc news at six                      5605
                                    ...  
through the lens: untold stories        2
rivals - america's endgame              2
off air                                 2
turner prize                            1
the firing line                         1
Length: 134, dtype: int64

In [542]:
q1_dummies = pd.DataFrame()
min_sample = 10

vc = q1[[x for x in q1.columns if "prog" in x]].stack().value_counts()

temp_df = pd.get_dummies(q1[[x for x in q1.columns if "prog" in x]].apply(lambda x: x.astype('category').cat.set_categories(list(vc[vc>=min_sample].index))))

cat_size = len(vc[vc>=min_sample].index)

for var_no in range(0,cat_size):
    amalg_name = temp_df.columns[var_no].replace("_before_yesterdayW18_1_prog","")
    q1_dummies[amalg_name] = temp_df[[temp_df.columns[(x*cat_size)+var_no] for x in range(9)]].any(axis=1)

In [544]:
q1_dummies.astype('uint8').to_csv('../BES_analysis_data/W19_comb_strings\\q1_dummied.csv')

In [543]:
q1_dummies

,q1_regional news and weather,q1_bbc news,q1_breakfast,q1_bbc news at ten,q1_bbc news at six,q1_bbc news at one,q1_itv evening news,q1_channel 4 news,q1_good morning britain,q1_press preview,...,q1_our world,q1_channel 4 news(amb),q1_itv news(amb),q1_pmq's,q1_sky news(amb),q1_party election broadcast on behalf of plaid cymru,q1_party election broadcast on behalf of the labour party,q1_your questions answered,q1_party election broadcast on behalf of the liberal democrats,q1_party election broadcast on behalf of the conservative party
id,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98448,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98449,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [238]:
# ## getting string data into a nice zip file

# dataset_name = "W19_comb_strings"
# data_subfolder = BES_data_folder + dataset_name + os.sep

# (manifest, dataset_filename, dataset_description, dataset_citation,
#  dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

# import pandas as pd
# import numpy as np
# # save csv manually with SPSSS

# csvfile = '../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.csv'
# encoding = "ISO-8859-1"
# df = pd.read_csv(csvfile,encoding=encoding)

# df.rename(columns={'ï»¿id':'id'},inplace=True)
# # bug screwing up character encoding of that file!

# df = df.set_index("id").sort_index()
# df = df.apply(lambda x: x.str.lower().replace(" ",np.nan).astype('category'))
# df.to_pickle('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.zip', compression='zip')

# # csv -> zip - loses all dtypes/structure ... but no interesting dtypes in the string dataset


In [549]:
search(BES_Panel,"wt_new")

wt_new_W10           30237
wt_new_W11           30956
wt_new_W14           31063
wt_new_W15           30842
wt_new_W16           37959
wt_new_W1_W16         2427
wt_new_W18           37825
wt_new_W6W7          21081
wt_new_W2W6          23163
wt_new_W2W6W7        17076
wt_new_W9_W13        12562
wt_new_W12           34394
wt_new_W1_W11         6473
wt_new_W1_W12         5833
wt_new_W1_W13         5301
wt_new_W6_W11         9373
wt_new_W6_W12         8398
wt_new_W6_W13         7622
wt_new_W11_W13       21520
wt_new_W13W16        14647
wt_new_W13_result    31135
wt_new_W6W16          9587
wt_new_W2W16          8388
wt_new_W17           34366
wt_new_W13W17        15744
wt_new_W6W13W19       8743
wt_new_W9W19         13604
wt_new_W13_W19        5861
wt_new_W13W19        15811
wt_new_W16W19        21840
wt_new_W17W18W19     19000
wt_new_W19_result    32177
wt_new_W9            30036
wt_new_W8            33502
wt_new_W7            30895
wt_new_W6            30073
wt_new_W13W14        20393
w

In [546]:
BES_Panel["p_past_vote_2019"]

id
1                 NaN
2        Conservative
3        Brexit Party
4                 NaN
5                 NaN
             ...     
98447             NaN
98448             NaN
98449             NaN
98450             NaN
98451             NaN
Name: p_past_vote_2019, Length: 97006, dtype: category
Categories (9, object): [Conservative < Labour < Liberal Democrat < Scottish National Party (SNP) ... Green Party < Other < Brexit Party < Don't know]

In [554]:
ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=BES_Panel["likeJohnsonW19"],
            values=BES_Panel["wt_new_W19_result"],normalize='columns',aggfunc=sum)*100).round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
likeJohnsonW19,,,,,,,,,
Strongly dislike,1.00,59.12,40.22,66.81,35.05,35.20,21.51,6.14,17.19
1,0.55,10.40,13.49,10.03,8.89,10.98,5.93,2.10,2.80
2,1.03,8.74,9.42,6.15,8.92,9.73,7.46,2.89,8.76
3,2.09,5.91,9.33,3.57,7.11,9.37,7.83,3.12,4.72
4,2.75,3.49,6.92,2.56,2.81,7.03,9.45,3.36,4.10
5,8.36,4.48,8.97,3.35,11.09,10.87,15.39,12.91,7.91
6,10.61,2.25,4.85,2.62,2.71,5.74,8.58,13.47,7.83
7,18.26,1.91,3.10,2.10,5.08,5.02,10.62,17.91,9.54
8,20.19,0.75,1.57,0.90,3.29,2.62,6.09,15.53,3.28


In [570]:
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)

In [562]:
sum(list(range(0,11)))

55

In [573]:
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Strongly dislike,1.00,59.12,40.22,66.81,35.05,35.20,21.51,6.14,17.19
1,0.55,10.40,13.49,10.03,8.89,10.98,5.93,2.10,2.80
2,1.03,8.74,9.42,6.15,8.92,9.73,7.46,2.89,8.76
3,2.09,5.91,9.33,3.57,7.11,9.37,7.83,3.12,4.72
4,2.75,3.49,6.92,2.56,2.81,7.03,9.45,3.36,4.10
5,8.36,4.48,8.97,3.35,11.09,10.87,15.39,12.91,7.91
6,10.61,2.25,4.85,2.62,2.71,5.74,8.58,13.47,7.83
7,18.26,1.91,3.10,2.10,5.08,5.02,10.62,17.91,9.54
8,20.19,0.75,1.57,0.90,3.29,2.62,6.09,15.53,3.28
9,12.43,0.24,0.36,0.49,8.98,0.26,1.89,9.07,0.79


In [574]:
ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=BES_Panel["likeConW19"],
            values=BES_Panel["wt_new_W19_result"],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Strongly dislike,0.77,58.09,31.02,70.34,34.25,34.93,26.50,10.56,20.13
1,0.39,8.87,8.85,7.73,8.13,8.78,4.74,3.46,4.32
2,0.76,9.75,10.80,6.01,6.43,8.85,8.63,4.42,5.47
3,1.56,7.64,11.20,5.09,14.87,9.98,8.33,5.61,4.02
4,2.35,4.34,8.60,2.00,7.11,6.40,9.28,5.18,2.78
5,11.07,5.31,12.14,3.12,12.79,13.07,18.84,17.98,15.51
6,10.64,1.88,6.74,2.29,4.04,5.17,8.21,11.94,3.65
7,19.35,1.22,5.69,1.64,2.76,5.07,6.43,15.48,3.64
8,21.06,0.41,1.88,0.26,0.80,2.96,2.62,11.71,2.73
9,10.80,0.14,0.43,0.00,0.00,0.00,0.73,6.09,0.47


In [576]:
ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=BES_Panel["ptvConW18"],
            values=BES_Panel["wt_new_W18"],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Very unlikely,1.32,63.77,36.90,82.80,27.63,44.96,32.94,18.37,14.09
1,0.27,5.53,6.68,1.98,2.32,5.90,2.61,3.44,7.66
2,0.69,5.86,7.68,2.37,8.61,11.79,7.38,2.77,0.44
3,0.62,4.33,6.36,2.09,0.00,5.01,3.69,3.14,0.00
4,0.79,2.63,5.97,1.08,0.00,9.68,0.52,4.84,0.00
5,3.84,3.77,8.06,2.23,6.67,2.55,17.34,9.42,3.35
6,3.46,2.56,8.59,0.93,1.63,4.36,6.72,9.72,6.35
7,9.02,1.99,6.29,2.34,5.51,6.56,9.58,4.07,3.44
8,10.64,1.98,2.70,0.47,15.65,2.73,7.03,10.58,5.63
9,10.26,0.16,1.16,0.00,4.24,1.23,2.63,2.26,0.00


In [579]:
BES_Panel["p_eurefvote"].value_counts()

Stay/remain in the EU    37525
Leave the EU             32516
Don't know                 179
Name: p_eurefvote, dtype: int64

In [581]:

BES_Panel["euRefVoteW18"].value_counts()

Stay/remain in the EU    17614
Leave the EU             16005
Don't know                2064
I would/will not vote     1760
Name: euRefVoteW18, dtype: int64

In [591]:
search(BES_Panel,"eurefVote")

euRefVoteW1                30529
euRefVoteW2                25817
euRefVoteW3                27830
euRefVoteW4                31550
euRefVoteW6                30258
euRefVoteW7                30895
euRefVoteSqueezeW7          4717
euRefVoteUnsqueezeW7       30895
euRefVotePostW8             4326
euRefVoteW8                33501
euRefVoteSqueezeW8          3818
euRefVoteUnsqueezeW8       29175
euRefVote2W8               29175
euRefVoteHypotheticalW9     1788
euRefVoteW9                28248
euRefVoteW10               30237
euRefVoteW11               30956
euRefVoteW12               34394
euRefVoteW13               31136
euRefVoteW14               31063
euRefVoteW15               30602
euRefVoteW16               37959
euRefVoteW17               34041
euRefVoteW18               37443
euRefVoteW19               32177
p_eurefvote                70220
dtype: int64

In [592]:
mask = BES_Panel["euRefVoteW18"]=="Leave the EU"

ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"][mask],
            index=BES_Panel["ptvConW18"][mask],
            values=BES_Panel["wt_new_W18"][mask],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Very unlikely,0.96,47.48,26.71,63.10,10.14,52.63,23.41,19.78,19.70
1,0.24,4.56,5.35,6.29,0.00,3.82,1.87,4.05,0.00
2,0.48,7.43,11.22,0.00,14.06,6.17,0.00,3.26,0.00
3,0.55,3.91,0.00,9.78,0.00,6.46,5.18,2.20,0.00
4,0.51,0.69,2.38,10.33,0.00,0.00,1.11,3.38,0.00
5,3.21,6.79,4.50,0.00,0.00,0.00,19.93,9.90,10.72
6,2.73,7.22,17.83,5.09,0.00,2.32,10.79,11.42,9.18
7,8.58,3.63,7.22,0.00,0.00,14.27,9.42,4.78,10.32
8,10.60,5.73,10.93,5.41,67.99,0.00,12.13,10.97,7.19
9,10.22,0.40,1.66,0.00,7.81,0.00,5.57,1.77,0.00


In [596]:
mask = BES_Panel["p_eurefvote"]=="Stay/remain in the EU"

ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"][mask],
            index=BES_Panel["ptvConW18"][mask],
            values=BES_Panel["wt_new_W18"][mask],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Very unlikely,1.92,68.65,40.97,85.06,58.27,46.41,39.61,10.13,26.40
1,0.35,5.77,7.91,1.92,6.00,8.29,2.58,0.00,0.00
2,0.90,6.22,7.58,3.11,8.40,8.83,8.00,0.00,0.00
3,1.27,3.89,6.58,1.22,0.00,3.97,2.33,13.90,0.00
4,0.61,2.66,5.99,0.31,0.00,6.48,1.41,0.00,0.00
5,6.02,2.53,8.47,3.00,0.00,3.55,17.13,13.14,3.86
6,3.53,1.57,7.00,0.74,4.21,6.78,2.51,0.00,9.47
7,10.49,1.93,5.71,1.80,4.63,6.31,12.54,0.00,3.44
8,12.01,1.78,2.05,0.00,9.50,4.70,1.42,0.00,0.00
9,10.00,0.09,1.11,0.00,3.55,0.00,1.08,9.98,0.00


In [603]:
mask = (BES_Panel["ageW18"]<25)&(BES_Panel["euRefVoteW18"]=="Stay/remain in the EU")

ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"][mask],
            index=BES_Panel["ptvConW18"][mask],
            values=BES_Panel["wt_new_W18"][mask],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Don't know
Very unlikely,0.00,65.10,8.25,100.0,0.0,29.32,0.00
1,0.00,3.33,3.48,0.0,0.0,0.00,66.49
2,0.00,6.33,15.88,0.0,0.0,10.03,0.00
3,0.00,3.57,5.16,0.0,0.0,0.00,0.00
4,0.00,5.71,23.03,0.0,0.0,35.58,0.00
5,0.00,2.21,13.45,0.0,0.0,0.00,0.00
6,9.30,3.43,20.25,0.0,0.0,25.07,0.00
7,39.47,2.34,0.00,0.0,0.0,0.00,0.00
8,31.00,2.59,0.00,0.0,0.0,0.00,33.51
9,0.00,0.00,4.24,0.0,0.0,0.00,0.00


In [606]:
(pd.crosstab(columns=BES_Panel["p_past_vote_2019"][mask],
            index=BES_Panel["ptvConW18"][mask],
            values=BES_Panel["wt_new_W18"][mask],aggfunc=sum)).round(2)

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Don't know
ptvConW18,,,,,,,
Very unlikely,NaN,144.26,4.08,28.59,NaN,4.48,NaN
1,NaN,7.39,1.72,NaN,NaN,NaN,5.56
2,NaN,14.04,7.86,NaN,NaN,1.53,NaN
3,NaN,7.91,2.56,NaN,NaN,NaN,NaN
4,NaN,12.65,11.39,NaN,NaN,5.44,NaN
5,NaN,4.89,6.66,NaN,NaN,NaN,NaN
6,0.80,7.61,10.02,NaN,NaN,3.83,NaN
7,3.41,5.18,NaN,NaN,NaN,NaN,NaN
8,2.68,5.74,NaN,NaN,NaN,NaN,2.80


In [611]:
pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=BES_Panel["ptvConW18"]).sum().sum()

5774

In [613]:
search(BES_Panel,"ptvCon")

ptvConW1        15198
ptvConW2        15216
ptvConW3        13971
ptvConW4        15855
ptvConW5        15527
ptvConW6        15250
ptvConW7        30895
ptvConW8         8966
ptvConW9        30036
ptvConW10       30237
ptvConW11       30956
ptvConW12        8515
ptvConW13       31136
ptvConExpW14     7939
ptvConW15       30842
ptvConW16       18756
ptvConW17       34366
ptvConW18        9439
dtype: int64

In [614]:
ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=BES_Panel["ptvConW17"],
            values=BES_Panel["wt_new_W17"],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Very unlikely,2.54,64.21,39.80,75.68,49.75,44.78,25.79,21.22,19.15
1,0.58,5.08,6.83,3.58,3.87,2.47,3.50,2.22,1.29
2,0.69,4.92,6.28,3.59,3.77,6.56,4.29,5.06,2.30
3,0.90,3.53,5.33,1.68,2.16,3.74,2.79,3.26,1.81
4,0.96,2.37,4.20,0.67,0.48,2.82,4.42,3.93,0.40
5,4.19,3.72,7.99,1.98,12.60,9.26,13.80,11.51,11.19
6,3.68,2.36,5.50,2.93,2.53,5.42,7.43,7.63,3.14
7,8.16,1.96,6.46,0.67,5.37,6.05,7.23,9.52,2.99
8,10.87,1.32,3.94,0.63,4.45,2.71,5.77,8.99,2.72
9,8.19,0.31,1.18,0.19,0.00,1.07,1.60,4.01,2.68


In [616]:
BES_Panel["p_past_vote_2019"].value_counts()

Conservative                     12695
Labour                            8264
Liberal Democrat                  3582
Scottish National Party (SNP)     1252
Green Party                        863
Brexit Party                       722
Other                              552
Don't know                         232
Plaid Cymru                        183
Name: p_past_vote_2019, dtype: int64

In [622]:
ptvCon_W17W18 = BES_Panel[["ptvConW17","ptvConW18"]].fillna(axis=1,method='ffill')["ptvConW18"]

In [ ]:
BES_Panel[["ptvConW17","ptvConW18"]].fillna(axis=1,method='ffill')

In [624]:
ptvCon_W17W18.value_counts()

Very unlikely    12369
Very likely       8490
Don't know        3621
8                 2199
7                 1965
5                 1861
9                 1490
6                 1318
2                 1185
1                 1167
3                 1012
4                  708
Name: ptvConW18, dtype: int64

In [621]:
BES_Panel[["ptvConW17","ptvConW18"]]

,ptvConW17,ptvConW18
id,,
1,NaN,NaN
2,8,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
...,...,...
98447,6,NaN
98448,Don't know,NaN
98449,1,NaN


In [625]:
ct = (pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=ptvCon_W17W18,
            values=BES_Panel["wt_new_W19_result"],normalize='columns',aggfunc=sum)*100).round(2)
ct.index = list(ct.index)
ct.loc["mean"] = [(ct.loc[ct.index[0:11],x]*pd.Series(range(0,11),index=ct.index[0:11])).sum()/100 for x in ct.columns]
ct=ct.round(2)
ct

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
Very unlikely,2.38,63.24,38.35,75.25,42.84,44.53,28.59,20.80,17.86
1,0.51,5.55,7.17,2.46,3.48,4.48,3.06,2.33,2.10
2,0.66,5.20,7.17,3.85,5.19,7.17,6.62,5.14,2.10
3,0.89,3.72,4.90,1.82,2.13,4.07,3.63,3.32,0.00
4,0.88,2.40,4.46,0.87,0.78,4.00,2.74,4.55,0.00
5,4.17,3.86,8.11,2.44,15.65,7.52,14.41,11.11,10.16
6,3.83,2.49,6.18,3.16,2.56,4.94,7.85,8.28,4.68
7,8.81,1.88,6.87,1.14,4.95,5.77,8.16,7.97,3.69
8,10.84,1.60,3.70,0.66,8.17,2.80,6.06,9.87,3.43
9,8.74,0.27,1.05,0.17,1.06,0.86,1.70,3.17,2.48


In [631]:
pd.crosstab(columns=BES_Panel["p_past_vote_2019"],
            index=ptvCon_W17W18,
            )

p_past_vote_2019,Conservative,Labour,Liberal Democrat,Scottish National Party (SNP),Plaid Cymru,Green Party,Other,Brexit Party,Don't know
ptvConW18,,,,,,,,,
Very unlikely,213,4386,1110,751,79,322,122,106,37
1,45,353,200,25,6,28,12,16,2
2,60,300,180,33,10,40,20,28,4
3,74,211,132,17,4,34,15,17,0
4,84,135,98,9,1,21,14,19,0
5,407,218,231,19,7,47,60,62,18
6,353,125,151,15,5,29,32,44,11
7,785,107,176,9,7,38,31,53,9
8,1056,87,104,5,8,15,24,59,9
